
# Joule Heating Statistics per Bin

## Introduction
Joule Heating is an observable which Daedalus satelite will be able to measure and is indicative of the phenomena we want to study. That is why we use it to check if Daedalus will be able to measure adequately the Thermosphere along his orbit.
Our purpose is to compare the sparse Joule Heating measurements which Daedalus will take while he passes along areas of interest with the dense measurements across a space-time grid of the whole area of interest.
The measurements data for the grid come from TIEGCM simulations for the 3 years of the mission (2015-7). The data measured along the orbit come from the same source, but are interpolated for each satellite position.
In order to compare the results we employ several plots and calculate mean, variance and standard deviation values.

#### Areas of interest
We have defined large areas called "regions" and we devide them in smaller ones called 'bins'. The bin boundaries are defined by ranges of:
1. Magnetic Local Time (MLT)
2. Magnetic Latitude (MagLat)
3. Altitude
4. Geomagnetic Kp index
For example, we have defined a region called "AEM - Auroral E region, midnight sector". AEM's boundaries are:
     60 <  Magnetic Latitude  < 75
  22:00 < Magnetic Local Time < 02:00
    115 < Altitude            < 140
      0 < Kp index            < 9
AEM is divided in 9 bins according to smaller ranges of altitude and Kp index.

## Data
#### TIEGCM grid data 
The Thermosphere is described in several TIEGCM files of netCDF type. Each file contains simulated data for 5 days and we have files for the satellite's 3 years lifetime. Inside the file there are data for every 2.5 degrees of Latitude, for every 2.5 degrees of Longitude, for every 2 hours and for 57 pressure levels.
#### Orbit data
A simulated orbit provides data about each position of the satellite stored in netCDF format. These include time, latitude, longitude, altitude Magnetic Latitude, Magnetic Local Time, Kp index and interpolated Joule Heating value.
#### Result data
The result data are stored in order to be ploted easier without the time intensive calculation.
The execution and ploting is separated in regions to make calculations easier to handle and plot more clear.
The result data are stored in both netCDF and plain text format and contain Joule Heating measurements across the area of interest. 


## Algorithms Description
Firstly, we use the TIEGCM grid data in order to have a picture of Joule Heating for each area of interest. 
A) For each area of interest:
     - parse all TIEGCM files and for every point of the space-time grid:
        - check if the point lies inside any of the pre-defined bins.
        - if it does, then assign the Joule Heating value of this point to the correct bin.
     - for each bin calculate the mean, variance, standard deviation of Joule Heating.
     - all the generated data constitute the TIEGCM-grid results
     
Afterwards we use the orbit data in order see the Joule Heating the satellite will be able to measure.
B) For each area of interest:
    - For every satellite position check if the satellite position lies inside any of the pre-defined bins:
		1. read Altitude, Magnetic-Latitude, Magnetic-Local-Time.
		2. Check if the above values lie inside the ranges of a bin.
	   	   If they do then we have to check the Kp-value following the next step. 		   
		4. Kp index is stored in a TIEGCM file. 
           Read the time of the satellite position and locate the corresponding TIEGCM file.
		5. Read the Kp-value according to the current time.
		6. Now we can check if the satellite position really lies inside a bin.
		   If it does, then assign the Joule Heating value to the correct bin.
    - for each bin calculate the mean, variance, standard deviation of Joule Heating.
    - all the generated data constitute the along-Orbit results


## Plots
We have constructed several plots to display the multi-dimensional data which result from the algorithm.
Both TIEGCM-grid and along-Orbit results are demonstrated using the same plots as described below.
#### Joule Heating versus Magnetic Latitude / Magnetic Local Time / Altitude
In these scatter plots each dot represents an instance of a measurement which was taken inside the area of interest. 
The plot usually does not display all the measurements because of their vast number. The plot also contains lines which indicate the Joule Heating mean and standard deviation calculated on all the values of the area of interest. The mean is displayed as a horizontal line and the standard deviation as a vertical line. 
The same plots are also available in a divided-by-Kp form. The Joule Heating versus Altiude plot display lines which connect the measurements which are successive along the orbit of the satellite (applies only on orbit data).
#### Joule Heating distribution
We provide a Joule Heating distribution plot for each area of interest and each bin. This plot can also display fitting functions along the data (An Euler function is usually the best fit).
#### Altitude versus Magnetic Latitude
There is also a scatter plot of Altitude versus Magnetic Latitude where each dot represents a measuremnt inside the area of interest and its color corresponds to its Joule Heating value.


In [3]:
import sys
sys.path.insert(1, '../../SourceCode/')
import DaedalusGlobals as DaedalusGlobals
import Conversions as Conversions

import os
from os import path
import statistics
import random 
import copy
from scipy.optimize import curve_fit

import csv
import glob
import math
import time
from datetime import datetime
from datetime import timezone
from dateutil.relativedelta import relativedelta
import calendar
import numpy as np 
import pandas as pd
import ipywidgets as w
import netCDF4
from netCDF4 import Dataset 
from numba import cuda
import threading

import plotly
import chart_studio.plotly as py 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.cm
import matplotlib.pyplot as matplt


# colors used at plotting
MyColors = ["#217ca3", "#e29930", "#919636", "#af1c1c", "#e7552c", "#1b4b5a", "#e4535e", "#aebd38", "#ffbb00", "#2c7873"]
def Hex_to_RGB(  HexColor ): # "#e29930" -->
    RGB = tuple(int(HexColor.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
    return str(RGB).strip('(').strip(')').strip()

# GUI elements with global scope
style1 = {'description_width':'170px'}
layout1 = {'width':'780px'}
style2 = {'description_width':'95px'}
layout2 = {'width':'160px'}
OrbitPreviewImage = w.Image( format='png', visible=False )
OrbitPreviewImage.layout.visibility = 'hidden'
ExecutionTitle_Text = w.Text(value="", description='Execution title:', style=style1, layout=layout1)
ExecutionDescr_Text = w.Text(value="", description='Execution description:', style=style1, layout=layout1)
Warning_HTML = w.HTML( value ="", color="Red", visible=False )
tiegcmFolder_Dropdown    = w.Dropdown( options=["/home/NAS/TIEGCM_DATA_2/TIEGCM_Lifetime_2015_to_2018_JH_QD/"], description='TIEGCM files: ', style=style1, layout=layout1)
BinGroups_Dropdown       = w.Dropdown( options=["AEM", "AFM", "AEE", "AED", "EEJ", "EPB", "SQ", "CF", "PCF"], description='Area of study: ', style=style1, layout=layout1)
#OrbitFilename_Dropdown  = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.Orbit_Files_Path + "DAED_ORB_Lifetime*.csv")), description='Along orbit filename: ', style=style1, layout=layout1)
OrbitFilesPath_Dropdown  = w.Dropdown( options=list(), description='Orbit Files Path: ', style=style1, layout=layout1)
SavedFilenames_Dropdown  = w.Dropdown( options=list(),  description='', style=style1, layout=layout1)
SavedFilenames2_Dropdown = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.CoverageResults_Files_Path + "*.ValuesPerBinResults.nc")),  description='', style=style1, layout=layout1)
Variable_DropDown         = w.Dropdown( options=["Joule Heating", "Electric Field North", "Electric Field East", "Pedersen Conductivity", "Hall Conductivity", "Convection Heating", "Wind Heating" ],  description='Variable', style=style1, layout=layout1)
Plot_JHvsMagLat_Checkbox       = w.Checkbox(value=True, description="Plot variable vs Magnetic Latitude", style=style1, layout=layout1 )
Plot_JHvsMLT_Checkbox          = w.Checkbox(value=True, description="Plot variable vs Magnetic Local Time", style=style1, layout=layout1 )
Plot_JHvsAltitude_Checkbox     = w.Checkbox(value=True, description="Plot variable vs Altitude", style=style1, layout=layout1 )
Plot_AltitudeVsMagLat_Checkbox = w.Checkbox(value=True, description="Plot Altitude vs Magnetic Latitude", style=style1, layout=layout1 )
Plot_JHdistribution_Checkbox   = w.Checkbox(value=True, description="Plot distribution per bin", style=style1, layout=layout1 )
Plot_AltitudeProfiles_Checkbox = w.Checkbox(value=True, description="Plot Altitude profiles", style=style1, layout=layout1 )
Plot_HeightIntegrated_Checkbox = w.Checkbox(value=True, description="Plot height-integated distribution (result filename just denotes the region)", style=style1, layout=layout1 )
RegressionOptions_Dropdown     = w.Dropdown( options=["None", "Polynomial - degree 1", "Polynomial - degree 2", "Polynomial - degree 3", "Polynomial - degree 4", "Polynomial - degree 5", "Polynomial - degree 6", "Power law", "Logarithmic", "Euler", "Maxwell"], value="Euler", description='Regression Analysis', style=style1, layout=layout1)

# set options for the saved result files
L = glob.glob(DaedalusGlobals.CoverageResults_Files_Path + "*.ValuesPerBinResults.nc")
L += glob.glob(DaedalusGlobals.CoverageResults_Files_Path + "*MultiFileResults/")
L += ["/home/balukid/old_onlyOhmic.TRO.TIEGCM_Lifetime_2015_to_2018_JH_QD.MultiFileResults/"]
L = sorted(L)
SavedFilenames_Dropdown.options = L

# set options for orbit locations
L = list()
L.append( "/home/NAS/Data_Files/InterpolatedData/TIEGCM_Lifetime_2015_to_2018_CAMP03/1HzIntepolatedDATA/" )
L.append( "/home/NAS/Data_Files/InterpolatedData/TIEGCM_Lifetime_2015_to_2018_CAMP02_115km/" )
OrbitFilesPath_Dropdown.options = L


# Properties of the current calculation
CALCULATIONS_Title = ""
CALCULATIONS_Description =""
CALCULATIONS_RegionName = ""
CALCULATIONS_OrbitFilesPath = ""
CALCULATIONS_ResultsFilename = ""
CALCULATIONS_TIEGCMfolder = ""
CALCULATIONS_ExecutionDuration = 0
SELECTED_VARIABLE           = ""
SELECTED_VARIABLE_longname  = ""
SELECTED_VARIABLE_shortname = ""
SELECTED_VARIABLE_units     = ""
# The following lists store data about each hit
all_JH_values       = list()
all_MagLat_values   = list()
all_MLT_values      = list()
all_Altitude_values = list()
all_Lat_values      = list()
all_Kp_values       = list() 
all_Time_values     = list()
all_HittedBin_IDs   = list()
all_EEX_values      = list()
all_EEY_values      = list()
all_Pedersen_values = list()
all_Hall_values     = list()
all_ConvectionHeating_values = list()
all_WindHeating_values = list()

# utility: converts a number to its 2-digit string representation
def num_to_2digit_str( n ):
    s = str(n)
    if len(s) == 1:
        s = '0' + s
    return s

# utility: takes a string containing numbers and places spaces instead of the leading zeros 
def ConvertLeadingZerosToSpaces( str ):
    result = ""
    leading_zone = True
    for c in str:
        if leading_zone:
            if c == '0':
                result = result + ' '
            else:
                result = result + c
                leading_zone = False
        else:
            result = result + c
    if result.strip().startswith('.')  and  result.startswith(' '): result = result[:result.rfind(' ')] + '0' + result.strip()
    if result.strip() == "": result = result[ :-1 ] + '0'
    if (result.startswith('.')) : result = '0' + result            
    return result

# Parses a string representing a date and returns a corresponding datetime object. Example: Jan 01 2015 00:01:10.000000000
def parseDaedalusDate( dateString ):
    result = None
    try:
        result = datetime.strptime(dateString[0:24], '%b %d %Y %H:%M:%S.%f')
    except:
        try:
            result = datetime.strptime(dateString, '%b %d %Y %H:%M:%S.%f')
        except:
            try:
                result = datetime.strptime(dateString, '%d %b %Y %H:%M:%S.%f')
            except:
                result = None
    return result
        

# utility: returns a color of a colormap as list of r,g,b,a values representing a value inside a range
def getColor( Value, minValue, maxValue, ColormapName ):
    cmap = matplotlib.cm.get_cmap( ColormapName )
    norm = matplotlib.colors.Normalize(vmin=minValue, vmax=maxValue)
    rgba = cmap( norm(Value) )
    s = "rgba" + str(rgba) 
    return s

# Define a class which can describe a bin
class Bin:
    ID             = ""
    Description    = ""
    MLT_min        = 0 # Magnetic Local Time (hour & min of the 24-hour day) (string)
    MLT_max        = 0 # Magnetic Local Time (hour & min of the 24-hour day) (string)
    MagLat_min     = 0 # Magnetic Latitude (degrees)
    MagLat_max     = 0 # Magnetic Latitude (degrees)
    Altitude_min   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Altitude_max   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Kp_min         = 0 #
    Kp_max         = 0 #
    Lat_min        = 0
    Lat_max        = 0
    NumOfBins      = 0 # How many parts will the Altitude range be splitted in
    CumulativeTime = 0 # (sec)
    DesirableCumulativeTime = 0 # (sec)
    JH_min      = 99999 # the minimum JH value inside the bin
    JH_max      = 0     # the maximum JH value inside the bin
    JH_mean     = 0 # the mean JH value inside the bin
    JH_variance = 0 # the variance of JH value inside the bin (variance = (1/(N-1)) * Sum{1->N}(X-MeanVariance)^2  )
    # Data:
    JH_values         = list() # here will be stored all Joule Heating values in order to calculate the variance at the end
    JH_distribution   = list() # the item 0 holds the number of points which have 0<JH<JH_max/100 etc
    MagLat_values     = list() #  these values correspond to the JH_values
    MLT_values        = list() #  these values correspond to the JH_values
    Altitude_values   = list() #  these values correspond to the JH_values
    Kp_values         = list() #  these values correspond to the JH_values
    Time_values       = list() #  these values correspond to the JH_values
    EEX_values        = list()
    EEY_values        = list()
    Pedersen_values   = list()
    Hall_values       = list()
    ConvectionHeating_values = list()
    WindHeating_values = list()
    
    def __init__(self, ID, Description, MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Lat_min, Lat_max, Kp_min, Kp_max, DesirableCumulativeTime):
        self.ID             = ID
        self.Description    = Description
        self.MLT_min        = MLT_min 
        self.MLT_max        = MLT_max
        self.MagLat_min     = MagLat_min
        self.MagLat_max     = MagLat_max
        self.Altitude_min   = Altitude_min
        self.Altitude_max   = Altitude_max
        self.Lat_min        = Lat_min
        self.Lat_max        = Lat_max                
        self.Kp_min         = Kp_min
        self.Kp_max         = Kp_max
        self.DesirableCumulativeTime = DesirableCumulativeTime
        self.JH_values       = list()
        self.JH_distribution = [0] * 100
        self.MagLat_values   = list()
        self.MLT_values      = list()
        self.Altitude_values = list()
        self.Lat_values       = list()
        self.Kp_values       = list()
        self.Time_values     = list()
        self.EEX_values        = list()
        self.EEY_values        = list()
        self.Pedersen_values   = list()
        self.Hall_values       = list()
        self.ConvectionHeating_values = list()
        self.WindHeating_values = list()

    def reset(self):
        self.JH_min      = 99999
        self.JH_mean     = 0
        self.JH_variance = 0
        self.JH_values       = list()
        self.MagLat_values   = list()
        self.MLT_values      = list()
        self.Altitude_values = list()
        self.Lat_values       = list()
        self.Kp_values       = list()
        self.Time_values     = list()
        self.EEX_values        = list()
        self.EEY_values        = list()
        self.Pedersen_values   = list()
        self.Hall_values       = list()        
        self.ConvectionHeating_values = list()
        self.WindHeating_values = list()
        
    def getInfo(self):
        s  = self.ID.ljust(8, ' ') + ": "
        s += "{:02.0f}".format(self.MLT_min)      + "<MLT<="    + "{:02.0f}".format(self.MLT_max)      + " "
        s += "{:03.0f}".format(self.MagLat_min)   + "<MagLat<=" + "{:03.0f}".format(self.MagLat_max)   + " "
        s += "{:03.0f}".format(self.Altitude_min) + "<Alt<="    + "{:03.0f}".format(self.Altitude_max) + " "
        s += str(self.Kp_min)             + "<Kp<="     + str(self.Kp_max)       + " "
        if self.JH_min == 99999:
            s += " JHmin=" + "         "
        else:
            s += " JHmin=" + "{:.3e}".format(self.JH_min) #ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.JH_min) )
        s += " JHmean=" + "{:.3e}".format(self.JH_mean) #ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.JH_mean) )
        s += " JHvariance=" + "{:.3e}".format(self.JH_variance) #ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.JH_variance) )
        ##
        str_JH = ""
        for i in range(0, len(self.JH_values) ):            
            str_JH += str( self.JH_values[i] )
            if i < len(self.JH_values)-1: str_JH += ','
        s += " JH_values=" + str_JH # ''.join(str(e) for e in self.JH_values)
        ##
        return s
    
    def printMe(self):
        print( self.getInfo()[:220] )


Bins = list() # this list holds the definitions of all bins
def InitializeBins():
    global Bins
    Bins = list()
    #                ID        Description                          MLT      MagLat    Altitude                Lat      Kp       DesiredTime(sec)
    Bins.append( Bin("AEM_L1", "Auroral E region, midnight sector", 21, 3,   60, 75,   115, 120,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L2", "Auroral E region, midnight sector", 21, 3,   60, 75,   120, 125,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L3", "Auroral E region, midnight sector", 21, 3,   60, 75,   125, 130,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L4", "Auroral E region, midnight sector", 21, 3,   60, 75,   130, 135,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L5", "Auroral E region, midnight sector", 21, 3,   60, 75,   135, 140,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L6", "Auroral E region, midnight sector", 21, 3,   60, 75,   140, 145,               -90,90,  0, 2,   50*60 ) )    
    Bins.append( Bin("AEM_L7", "Auroral E region, midnight sector", 21, 3,   60, 75,   145, 150,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEM_M1", "Auroral E region, midnight sector", 21, 3,   60, 75,   115, 122,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEM_M2", "Auroral E region, midnight sector", 21, 3,   60, 75,   122, 129,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEM_M3", "Auroral E region, midnight sector", 21, 3,   60, 75,   129, 136,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEM_M4", "Auroral E region, midnight sector", 21, 3,   60, 75,   136, 143,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEM_M5", "Auroral E region, midnight sector", 21, 3,   60, 75,   143, 150,               -90,90,  2, 4,   30*60 ) )    
    Bins.append( Bin("AEM_H1", "Auroral E region, midnight sector", 21, 3,   60, 75,   115, 150,               -90,90,  4, 9,   20*60 ) )

    Bins.append( Bin("AEL_L0", "Auroral E region, midnight sector", 21, 3,   60, 75,    95, 115,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_L1", "Auroral E region, midnight sector", 21, 3,   60, 75,   115, 120,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_L2", "Auroral E region, midnight sector", 21, 3,   60, 75,   120, 125,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_L3", "Auroral E region, midnight sector", 21, 3,   60, 75,   125, 130,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_L4", "Auroral E region, midnight sector", 21, 3,   60, 75,   130, 135,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_L5", "Auroral E region, midnight sector", 21, 3,   60, 75,   135, 140,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_L6", "Auroral E region, midnight sector", 21, 3,   60, 75,   140, 145,               -90,90,  0, 2,   50*60 ) )    
    Bins.append( Bin("AEL_L7", "Auroral E region, midnight sector", 21, 3,   60, 75,   145, 150,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEL_M0", "Auroral E region, midnight sector", 21, 3,   60, 75,    95,            115,    -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEL_M1", "Auroral E region, midnight sector", 21, 3,   60, 75,   115,       126.6666,    -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEL_M2", "Auroral E region, midnight sector", 21, 3,   60, 75,   126.6666,  138.3333,    -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEL_M3", "Auroral E region, midnight sector", 21, 3,   60, 75,   138.33333, 150,         -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEL_H0", "Auroral E region, midnight sector", 21, 3,   60, 75,    95, 115,               -90,90,  4, 9,   20*60 ) )    
    Bins.append( Bin("AEL_H1", "Auroral E region, midnight sector", 21, 3,   60, 75,   115, 150,               -90,90,  4, 9,   20*60 ) )    
    
    Bins.append( Bin("AFM_L1", "Auroral F region, midnight sector", 21, 3,   60, 75,   150, 185,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L2", "Auroral F region, midnight sector", 21, 3,   60, 75,   185, 220,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L3", "Auroral F region, midnight sector", 21, 3,   60, 75,   220, 255,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L4", "Auroral F region, midnight sector", 21, 3,   60, 75,   255, 290,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L5", "Auroral F region, midnight sector", 21, 3,   60, 75,   290, 325,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L6", "Auroral F region, midnight sector", 21, 3,   60, 75,   325, 360,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L7", "Auroral F region, midnight sector", 21, 3,   60, 75,   360, 395,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L8", "Auroral F region, midnight sector", 21, 3,   60, 75,   395, 430,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L9", "Auroral F region, midnight sector", 21, 3,   60, 75,   430, 465,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L10","Auroral F region, midnight sector", 21, 3,   60, 75,   465, 500,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AFM_M1", "Auroral F region, midnight sector", 21, 3,   60, 75,   150.0, 237.5,           -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AFM_M2", "Auroral F region, midnight sector", 21, 3,   60, 75,   237.5, 325.0,           -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AFM_M3", "Auroral F region, midnight sector", 21, 3,   60, 75,   325.0, 412.5,           -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AFM_M4", "Auroral F region, midnight sector", 21, 3,   60, 75,   412.5, 500.0,           -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AFM_H1", "Auroral F region, midnight sector", 21, 3,   60, 75,   150, 265,               -90,90,  4, 9,   20*60 ) )
    Bins.append( Bin("AFM_H2", "Auroral F region, midnight sector", 21, 3,   60, 75,   265, 380,               -90,90,  4, 9,   20*60 ) )
    Bins.append( Bin("AFM_H3", "Auroral F region, midnight sector", 21, 3,   60, 75,   380, 500,               -90,90,  4, 9,   20*60 ) )
    
    Bins.append( Bin("AEE_L1", "Auroral E region, evening sector",  15, 21,  60, 75,   115, 120,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L2", "Auroral E region, evening sector",  15, 21,  60, 75,   120, 125,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L3", "Auroral E region, evening sector",  15, 21,  60, 75,   125, 130,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L4", "Auroral E region, evening sector",  15, 21,  60, 75,   130, 135,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L5", "Auroral E region, evening sector",  15, 21,  60, 75,   135, 140,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L6", "Auroral E region, evening sector",  15, 21,  60, 75,   140, 145,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L7", "Auroral E region, evening sector",  15, 21,  60, 75,   145, 150,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AEE_M1", "Auroral E region, evening sector",  15, 21,  60, 75,   115, 122,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEE_M2", "Auroral E region, evening sector",  15, 21,  60, 75,   122, 129,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEE_M3", "Auroral E region, evening sector",  15, 21,  60, 75,   129, 136,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEE_M4", "Auroral E region, evening sector",  15, 21,  60, 75,   136, 143,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEE_M5", "Auroral E region, evening sector",  15, 21,  60, 75,   143, 150,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AEE_H1", "Auroral E region, evening sector",  15, 21,  60, 75,   115, 150,               -90,90,  4, 9,   20*60 ) )

    Bins.append( Bin("AED_L1", "Auroral E region, dawn sector",     3, 9,   60, 75,   115, 120,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_L2", "Auroral E region, dawn sector",     3, 9,   60, 75,   120, 125,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_L3", "Auroral E region, dawn sector",     3, 9,   60, 75,   125, 130,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_L4", "Auroral E region, dawn sector",     3, 9,   60, 75,   130, 135,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_L5", "Auroral E region, dawn sector",     3, 9,   60, 75,   135, 140,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_L6", "Auroral E region, dawn sector",     3, 9,   60, 75,   140, 145,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_L7", "Auroral E region, dawn sector",     3, 9,   60, 75,   145, 150,                 -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("AED_M1", "Auroral E region, dawn sector",     3, 9,   60, 75,   115, 122,                 -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AED_M2", "Auroral E region, dawn sector",     3, 9,   60, 75,   122, 129,                 -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AED_M3", "Auroral E region, dawn sector",     3, 9,   60, 75,   129, 136,                 -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AED_M4", "Auroral E region, dawn sector",     3, 9,   60, 75,   136, 143,                 -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AED_M5", "Auroral E region, dawn sector",     3, 9,   60, 75,   143, 150,                 -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("AED_H1", "Auroral E region, dawn sector",     3, 9,   60, 75,   115, 150,                 -90,90,  4, 9,   20*60 ) )
    
    Bins.append( Bin("EEJ_A1", "Equatorial E-region",             10, 13,  -7,  7,   115,   127,                -90,90,  0, 9,   10*60 ) )
    Bins.append( Bin("EEJ_A2", "Equatorial E-region",             10, 13,  -7,  7,   127,   139,                -90,90,  0, 9,   10*60 ) )
    Bins.append( Bin("EEJ_A3", "Equatorial E-region",             10, 13,  -7,  7,   139,   150,                -90,90,  0, 9,   10*60 ) )

    Bins.append( Bin("EPB_A1", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   150, 185,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A2", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   185, 220,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A3", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   220, 255,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A4", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   255, 290,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A5", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   290, 325,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A6", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   325, 360,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A7", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   360, 395,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A8", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   395, 430,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A9", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   430, 465,                  -90,90,  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A10","Equatorial Plasma Bubbles",       18,  4, -30, 30,   465, 500,                  -90,90,  0, 9,   150*60 ) )

    Bins.append( Bin("SQ_A1",  "Sq & midlat F region currents",    6, 19, -60, 60,   150, 185,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A2",  "Sq & midlat F region currents",    6, 19, -60, 60,   185, 220,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A3",  "Sq & midlat F region currents",    6, 19, -60, 60,   220, 255,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A4",  "Sq & midlat F region currents",    6, 19, -60, 60,   255, 290,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A5",  "Sq & midlat F region currents",    6, 19, -60, 60,   290, 325,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A6",  "Sq & midlat F region currents",    6, 19, -60, 60,   325, 360,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A7",  "Sq & midlat F region currents",    6, 19, -60, 60,   360, 395,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A8",  "Sq & midlat F region currents",    6, 19, -60, 60,   395, 430,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A9",  "Sq & midlat F region currents",    6, 19, -60, 60,   430, 465,                  -90,90,  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A10", "Sq & midlat F region currents",    6, 19, -60, 60,   465, 500,                  -90,90,  0, 3,   150*60 ) )
    
    Bins.append( Bin("CF_L1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 185,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L2", "Dayside Cusp F-region",            10, 14,   70,  80,   185, 230,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L3", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 275,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L4", "Dayside Cusp F-region",            10, 14,   70,  80,   275, 320,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L5", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 365,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L6", "Dayside Cusp F-region",            10, 14,   70,  80,   365, 410,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L7", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 455,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_L8", "Dayside Cusp F-region",            10, 14,   70,  80,   455, 500,                -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("CF_M1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 230,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("CF_M2", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 320,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("CF_M3", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 410,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("CF_M4", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 500,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("CF_H1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 260,               -90,90,  4, 9,   20*60 ) )
    Bins.append( Bin("CF_H2", "Dayside Cusp F-region",            10, 14,   70,  80,   260, 380,               -90,90,  4, 9,   20*60 ) )
    Bins.append( Bin("CF_H3", "Dayside Cusp F-region",            10, 14,   70,  80,   380, 500,               -90,90,  4, 9,   20*60 ) )
    
    Bins.append( Bin("PCF_L1", "Polar cap F-region",              14, 10,   70,  90,   140, 185,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L2", "Polar cap F-region",              14, 10,   70,  90,   185, 230,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L3", "Polar cap F-region",              14, 10,   70,  90,   230, 275,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L4", "Polar cap F-region",              14, 10,   70,  90,   275, 320,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L5", "Polar cap F-region",              14, 10,   70,  90,   320, 365,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L6", "Polar cap F-region",              14, 10,   70,  90,   365, 410,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L7", "Polar cap F-region",              14, 10,   70,  90,   410, 455,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L8", "Polar cap F-region",              14, 10,   70,  90,   455, 500,               -90,90,  0, 2,   50*60 ) )
    Bins.append( Bin("PCF_M1", "Polar cap F-region",              14, 10,   70,  90,   140, 230,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("PCF_M2", "Polar cap F-region",              14, 10,   70,  90,   230, 320,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("PCF_M3", "Polar cap F-region",              14, 10,   70,  90,   320, 410,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("PCF_M4", "Polar cap F-region",              14, 10,   70,  90,   410, 500,               -90,90,  2, 4,   30*60 ) )
    Bins.append( Bin("PCF_H1", "Polar cap F-region",              14, 10,   70,  90,   140, 260,               -90,90,  4, 9,   20*60 ) )
    Bins.append( Bin("PCF_H2", "Polar cap F-region",              14, 10,   70,  90,   260, 380,               -90,90,  4, 9,   20*60 ) )
    Bins.append( Bin("PCF_H3", "Polar cap F-region",              14, 10,   70,  90,   380, 500,               -90,90,  4, 9,   20*60 ) )
    
    Bins.append( Bin("TRO_02", "EISCAT Tromso radar scan region",  0, 24,  -90,  90,   80, 150,                 60,90,  0, 2,   20*60 ) )
    Bins.append( Bin("TRO_04", "EISCAT Tromso radar scan region",  0, 24,  -90,  90,   80, 150,                 60,90,  2, 4,   20*60 ) )
    Bins.append( Bin("TRO_09", "EISCAT Tromso radar scan region",  0, 24,  -90,  90,   80, 150,                 60,90,  4, 9,   20*60 ) )
    #Bins.append( Bin("TRO_00", "EISCAT Tromso radar scan region",  0, 24,  -90,  90,   80, 150,                 60,90,  0, 9,   20*60 ) )
    
    #Bins.append( Bin("TST_00", "Test region",                      0, 24,  -90,  90,   80, 150,                 60,90,  0, 9,   20*60 ) )
    binGroupNames = list()
    for B in Bins:
        aGroupName = B.ID[ : B.ID.find("_") ]
        if aGroupName not in binGroupNames: binGroupNames.append( aGroupName )
    BinGroups_Dropdown.options = binGroupNames
    
InitializeBins()

# tries to identify the bin according to the given argument and returns its description. If it fails it returns the argument.
# examples: "PCF_H2"->"Polar cap F-region"   "PCF"->"Polar cap F-region"
def getBinDescription( str ):
    result = ""
    for B in Bins:
        if B.ID == str: result = B.Description
    if len(result)==0:
        for B in Bins:
            if B.ID.startswith( str ): result = B.Description
    if len(result)==0: result = str
    #
    return result


def createBinsForTheWholeEarth():
    n = 1
    for Kp_min in [0, 2, 4]:
        if Kp_min == 0:
            Kp_max = 2
        elif Kp_min == 2:
            Kp_max = 4
        elif Kp_min == 4:
            Kp_max = 9
        ####matc
        for MLT_min in range(0, 24, 4):
            for MagLat_min in range(-180, 180, 20):
                for Alt_min in range(115, 250, 25):
                    n = n + 1
                    Bins.append( Bin("E"+str(n), "",            MLT_min, MLT_min+4,   MagLat_min, MagLat_min+15,   Alt_min, Alt_min+25,               Kp_min, Kp_max,   20*60 ) )                    
    print ( len(Bins) + " Bins covering the whole Earth.")    
#createBinsForTheWholeEarth()

# cheks if certain MLT lies in a certain range. Created in order to take account ranges like 22-2
def is_MLT_inside_range( MLT, MLT_min, MLT_max ):
    if MLT_min <= MLT_max: # example: from 13 to 18 hour
        return (MLT > MLT_min  and  MLT <= MLT_max)
    else: # example: from 22 to 3 hour
        return (MLT > MLT_min  or   MLT <= MLT_max)

# returns: the bin object which matches the arguments
def GetMatchedBin( MLT, MagLat, Altitude, Kp, Latitude ):
    MatchedBin = None
    for B in Bins:
        if Latitude >= B.Lat_min  and  Latitude <= B.Lat_max:
            if is_MLT_inside_range(MLT, B.MLT_min, B.MLT_max):
                if MagLat   > B.MagLat_min    and  MagLat   <= B.MagLat_max:
                    if Altitude > B.Altitude_min  and  Altitude <= B.Altitude_max:
                        Kp_min_to_check = B.Kp_min
                        if Kp_min_to_check == 0: Kp_min_to_check = -1
                        if Kp       > Kp_min_to_check and  Kp       <= B.Kp_max:
                            MatchedBin = B
                            break
    return MatchedBin


# returns: the bin object which matches the arguments
def getBinByItsProperties( MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max ):
    CorrectBin = None
    for B in Bins:
        if             MLT_min      == B.MLT_min       and  MLT_max      == B.MLT_max:
            if         MagLat_min   == B.MagLat_min    and  MagLat_max   == B.MagLat_max:
                if     Altitude_min == B.Altitude_min  and  Altitude_max == B.Altitude_max:
                    if Kp_min       == B.Kp_min        and  Kp_max       == B.Kp_max:
                        CorrectBin = B
                        break
    return CorrectBin

# returns: the bin object which matches the arguments
def getBinByItsID( aBinID ):
    CorrectBin = None
    for B in Bins:
        if  B.ID == aBinID:
            CorrectBin = B
            break
    return CorrectBin


        
# Save the results in a text file        
def SaveResults_TXT( ResultsFilename ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilesPath, CALCULATIONS_TIEGCMfolder, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    nowstr = datetime.now().strftime("%d-%m-%Y %H:%M:%S")    
    F = open(ResultsFilename, 'w')
    F.write( "# -- JOULE HEATING per BIN RESULTS -- " + "\n"  )
    F.write( "# Date of execution: " + nowstr + "\n" )
    F.write( "# Title: " + CALCULATIONS_Title + "\n" )
    F.write( "# Region: " + CALCULATIONS_RegionName + "\n" )
    F.write( "# Orbit Filename: " + CALCULATIONS_OrbitFilesPath + "\n" )
    F.write( "# Description: " + CALCULATIONS_Description + "\n")
    F.write( "# DataPath: " + CALCULATIONS_TIEGCMfolder + "\n")
    F.write( "# Duration of execution: " + ConvertLeadingZerosToSpaces("{0:.0f}".format(CALCULATIONS_ExecutionDuration)) + " seconds  or  " + ConvertLeadingZerosToSpaces("{0:.2f}".format(CALCULATIONS_ExecutionDuration/60))  + " minutes" + "\n" )
    F.write( "# " + "\n")    
    for B in Bins:
        F.write( B.getInfo() + "\n" )
    ##
    F.write( "\nAll JH values: " ) 
    for i in range(0, len(all_JH_values) ):
        F.write( str( all_JH_values[i]) )
        if i < len(all_JH_values)-1: F.write( ',' )
    F.write( "\nAll MagLat values: " ) 
    for i in range(0, len(all_MagLat_values) ):
        F.write( "{:.4g}".format( all_MagLat_values[i]) )
        if i < len(all_MagLat_values)-1: F.write( ',' )   
    F.write( "\nAll MLT values: " ) 
    for i in range(0, len(all_MLT_values) ):
        F.write( "{:.4g}".format( all_MLT_values[i]) )
        if i < len(all_MLT_values)-1: F.write( ',' ) 
    F.write( "\nAll Altitude values: " ) 
    for i in range(0, len(all_Altitude_values) ):
        F.write( "{:.4g}".format( all_Altitude_values[i]) )
        if i < len(all_Altitude_values)-1: F.write( ',' )             
    ## write all data separated at bins
    F.write("\n")
    for B in Bins:
        F.write("\n")
        F.write( "BIN " + B.ID + ": MagLat values = " )
        for i in range(0, len(B.MagLat_values) ):
            F.write(  "{:.5g}".format(B.MagLat_values[i])  )
            if i < len(B.MagLat_values)-1: F.write( ',' )
        F.write("\n")
        F.write( "BIN " + B.ID + ": MLT values = " )
        for i in range(0, len(B.MLT_values) ):
            F.write(  "{:.5g}".format(B.MLT_values[i])  )
            if i < len(B.MLT_values)-1: F.write( ',' )
        F.write("\n")
        F.write( "BIN " + B.ID + ": Altitude values = " )
        for i in range(0, len(B.Altitude_values) ):
            F.write(  "{:.5g}".format(B.Altitude_values[i])  )
            if i < len(B.Altitude_values)-1: F.write( ',' )
    ##
    F.close()
    
    
    
    
    
# creates a results NetCDF file and its structure
def CreateResults_CDF( ResultsFilename ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilesPath, CALCULATIONS_TIEGCMfolder, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values, all_Kp_values, all_Time_values, all_EEX_values, all_EEY_values, all_Pedersen_values, all_Hall_values
    # save general info
    resultsCDF = Dataset( ResultsFilename, 'w' )
    resultsCDF.Content         = "JOULE HEATING per BIN RESULTS. This file contains information about the bins in which the thermosphere is divided according to Magnetic Latitude, Magnetic Local Time, Altitude and Kp-index. We say there is a hit inside a bin when a satellite position or TIEGCM-grid position lies inside the above boundaries. The file contains data for each hit inside a bin. That is the position's MagLat, MLT, Alt, Kp and Joule-Heating value"
    resultsCDF.DateOfCreation  = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    resultsCDF.DateOfUpdate    = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    resultsCDF.Title           = CALCULATIONS_Title
    resultsCDF.Region          = CALCULATIONS_RegionName
    resultsCDF.OrbitFile       = CALCULATIONS_OrbitFilesPath
    resultsCDF.Description     = CALCULATIONS_Description
    resultsCDF.DataPath        = CALCULATIONS_TIEGCMfolder
    resultsCDF.LastExecDurationSec = 0
    resultsCDF.Progress        = ""
    # save data for each bin spearately 
    resultsCDF.createDimension( "SingleSpaceFooDimension", 1 )
    resultsCDF.createDimension('char8', 8)
    for B in Bins:
        # save general info about the bin
        VAR_BinInfo = resultsCDF.createVariable( B.ID, "S1", ("SingleSpaceFooDimension",) )
        VAR_BinInfo.long_name    = "Information about the bin " + B.ID + " (" + B.Description + ")"
        VAR_BinInfo.MagLat_min   = "{:02.0f}".format(B.MagLat_min)
        VAR_BinInfo.MagLat_max   = "{:02.0f}".format(B.MagLat_max)
        VAR_BinInfo.MLT_min      = "{:02.0f}".format(B.MLT_min)
        VAR_BinInfo.MLT_max      = "{:02.0f}".format(B.MLT_max)
        VAR_BinInfo.Altitude_min = "{:02.0f}".format(B.Altitude_min)
        VAR_BinInfo.Altitude_max = "{:02.0f}".format(B.Altitude_max)
        VAR_BinInfo.Lat_min       = "{:02.0f}".format(B.Lat_min)
        VAR_BinInfo.Lat_max       = "{:02.0f}".format(B.Lat_max)
        VAR_BinInfo.Kp_min       = "{:02.0f}".format(B.Kp_min)
        VAR_BinInfo.Kp_max       = "{:02.0f}".format(B.Kp_max)
        VAR_BinInfo.JH_mean      = "{:.3e}".format(B.JH_mean)
        VAR_BinInfo.JH_variance  = "{:.3e}".format(B.JH_variance)
        VAR_BinInfo.DesirableCumulativeTime = str(B.DesirableCumulativeTime) + "sec"
        if B.JH_min == 99999: 
            VAR_BinInfo.JH_min = ""
        else:
            VAR_BinInfo.JH_min = "{:.3e}".format(B.JH_min)
        # create structure for each bin
        resultsCDF.createDimension( B.ID+"_time_dim", None )
        VAR_BinTimeValues             = resultsCDF.createVariable( B.ID+"_TimeValues", "f4", (B.ID+"_time_dim",) )
        VAR_BinTimeValues.description = "UTC timestamp"
        VAR_BinTimeValues.units       = "seconds"
        resultsCDF.createDimension( B.ID+"_jh_dim", None )
        VAR_BinJHvalues = resultsCDF.createVariable( B.ID+"_JHValues", "f4", (B.ID+"_jh_dim",) )
        VAR_BinJHvalues.description = "Ohmic"
        VAR_BinJHvalues.units       = "W/m3"
        resultsCDF.createDimension( B.ID+"_maglat_dim", None )
        VAR_BinMagLatValues = resultsCDF.createVariable( B.ID+"_MagLatValues", "f4", (B.ID+"_maglat_dim",) )
        VAR_BinMagLatValues.description = "Magnetic Latitude"
        VAR_BinMagLatValues.units       = "degrees"
        resultsCDF.createDimension( B.ID+"_mlt_dim", None )
        VAR_BinMLTValues = resultsCDF.createVariable( B.ID+"_MLTValues", "f4", (B.ID+"_mlt_dim",) )
        VAR_BinMLTValues.description = "Magnetic Local Time"
        VAR_BinMLTValues.units       = "hours"
        resultsCDF.createDimension( B.ID+"_alt_dim", None )
        VAR_BinAltitudeValues = resultsCDF.createVariable( B.ID+"_AltitudeValues", "f4", (B.ID+"_alt_dim",) )
        VAR_BinAltitudeValues.description = "Altitude from the surface of the Earth"
        VAR_BinAltitudeValues.units       = "km"
        resultsCDF.createDimension( B.ID+"_lat_dim", None )
        VAR_BinLatValues = resultsCDF.createVariable( B.ID+"_LatValues", "f4", (B.ID+"_lat_dim",) )
        VAR_BinLatValues.description = "Latitude"
        VAR_BinLatValues.units       = "degrees"
        resultsCDF.createDimension( B.ID+"_kp_dim", None )
        VAR_BinKpValues = resultsCDF.createVariable( B.ID+"_KpValues", "f4", (B.ID+"_kp_dim",) )
        VAR_BinKpValues.description = "Kp index of Sun activity"
        VAR_BinKpValues.units       = "-"
        resultsCDF.createDimension( B.ID+"_eex_dim", None )
        VAR_BinEEXValues = resultsCDF.createVariable( B.ID+"_EEXValues", "f4", (B.ID+"_eex_dim",) )
        VAR_BinEEXValues.description = "Electric field strength East. (SI)"
        VAR_BinEEXValues.units       = "V/m"
        resultsCDF.createDimension( B.ID+"_eey_dim", None )
        VAR_BinEEYValues = resultsCDF.createVariable( B.ID+"_EEYValues", "f4", (B.ID+"_eey_dim",) )
        VAR_BinEEYValues.description = "Electric field strength North. (SI)"
        VAR_BinEEYValues.units       = "V/m"
        resultsCDF.createDimension( B.ID+"_ped_dim", None )
        VAR_BinPedersenValues = resultsCDF.createVariable( B.ID+"_PedersenValues", "f4", (B.ID+"_ped_dim",) )
        VAR_BinPedersenValues.description = "SIGMA_PED"
        VAR_BinPedersenValues.units       = "S/m"
        resultsCDF.createDimension( B.ID+"_hal_dim", None )
        VAR_BinHallValues = resultsCDF.createVariable( B.ID+"_HallValues", "f4", (B.ID+"_hal_dim",) )
        VAR_BinHallValues.description = "SIGMA_HAL"
        VAR_BinHallValues.units       = "S/m"
        resultsCDF.createDimension( B.ID+"_convh_dim", None )
        VAR_BinConvhValues = resultsCDF.createVariable( B.ID+"_ConvectionHeatingValues", "f4", (B.ID+"_convh_dim",) )
        VAR_BinConvhValues.description = "Convection Heating"
        VAR_BinConvhValues.units       = "W/m3"
        resultsCDF.createDimension( B.ID+"_windh_dim", None )
        VAR_BinWindhValues = resultsCDF.createVariable( B.ID+"_WindHeatingValues", "f4", (B.ID+"_windh_dim",) )
        VAR_BinWindhValues.description = "Wind Heating"
        VAR_BinWindhValues.units       = "W/m3"
    ## save data for all hits
    resultsCDF.createDimension( "time_dim", None )
    VAR_TimeValues         = resultsCDF.createVariable("allTimeValues", "f4", ("time_dim",) )
    VAR_TimeValues.description = "UTC timestamp"
    VAR_TimeValues.units       = "seconds"
    resultsCDF.createDimension( "jh_dim", None )
    VAR_JHvalues = resultsCDF.createVariable("allJHValues", "f4", ("jh_dim",) )
    VAR_JHvalues.description = "Ohmic"
    VAR_JHvalues.units       = "W/m3"
    resultsCDF.createDimension( "maglat_dim", None )
    VAR_MagLatValues = resultsCDF.createVariable("allMagLatValues", "f4", ("maglat_dim",) )
    VAR_MagLatValues.description = "Magnetic Latitude"
    VAR_MagLatValues.units       = "degrees"
    resultsCDF.createDimension( "mlt_dim", None )
    VAR_MLTValues = resultsCDF.createVariable("allMLTValues", "f4", ("mlt_dim",) )
    VAR_MLTValues.description = "Magnetic Local Time"
    VAR_MLTValues.units       = "hours"
    resultsCDF.createDimension( "alt_dim", None )
    VAR_AltitudeValues = resultsCDF.createVariable("allAltitudeValues", "f4", ("alt_dim",) )
    VAR_AltitudeValues.description = "Altitude from the surface of the Earth"
    VAR_AltitudeValues.units       = "km"
    resultsCDF.createDimension( "lat_dim", None )
    VAR_LatValues = resultsCDF.createVariable("allLatValues", "f4", ("lat_dim",) )
    VAR_LatValues.description = "Latitude"
    VAR_LatValues.units       = "degrees"
    resultsCDF.createDimension( "kp_dim", None )
    VAR_KpValues = resultsCDF.createVariable("allKpValues", "f4", ("kp_dim",) )
    VAR_KpValues.description = "Kp index of Sun activity"
    VAR_KpValues.units       = "-"
    resultsCDF.createDimension( "bins_dim", None )
    VAR_HittedBinIDs = resultsCDF.createVariable("allHittedBinIDs", "S1", ("bins_dim","char8",) )
    VAR_HittedBinIDs.description = "The ID of the bin, where the hit occured"
    resultsCDF.createDimension( "eex_dim", None )
    VAR_EEXvalues = resultsCDF.createVariable("allEEXValues", "f4", ("eex_dim",) )
    VAR_EEXvalues.description = "Electric field strength East. (SI)"
    VAR_EEXvalues.units       = "V/m"
    resultsCDF.createDimension( "eey_dim", None )
    VAR_EEYvalues = resultsCDF.createVariable("allEEYValues", "f4", ("eey_dim",) )
    VAR_EEYvalues.description = "Electric field strength North. (SI)"
    VAR_EEYvalues.units       = "V/m"
    resultsCDF.createDimension( "ped_dim", None )
    VAR_Pedersenvalues = resultsCDF.createVariable("allPedersenValues", "f4", ("ped_dim",) )
    VAR_Pedersenvalues.description = "Pedersen Conductivity"
    VAR_Pedersenvalues.units       = "S/m"
    resultsCDF.createDimension( "hal_dim", None )
    VAR_Hallvalues = resultsCDF.createVariable("allHallValues", "f4", ("hal_dim",) )
    VAR_Hallvalues.description = "Hall Conductivity"
    VAR_Hallvalues.units       = "S/m"
    resultsCDF.createDimension( "convh_dim", None )
    VAR_ConvhValues = resultsCDF.createVariable("allConvectionHeatingValues", "f4", ("convh_dim",) )
    VAR_ConvhValues.description = "Convection Heating"
    VAR_ConvhValues.units       = "W/m3"
    resultsCDF.createDimension( "windh_dim", None )
    VAR_WindhValues = resultsCDF.createVariable("allWindHeatingValues", "f4", ("windh_dim",) )
    VAR_WindhValues.description = "Wind Heating"
    VAR_WindhValues.units       = "W/m3"
    resultsCDF.close()
    
    
# Append the results in a NetCDF file. The data will be saved in ResultsFilename and they come from DataFilename.
# DataFilename is needed to check if the file contains already the results of the DataFilename
def SaveResults_CDF( ResultsFilename, DataFilename ):
    if path.exists( ResultsFilename ) == False:
        CreateResults_CDF( ResultsFilename )
    # save general info
    ErrorMsg = ""
    resultsCDF = Dataset( ResultsFilename, 'a' )
    resultsCDF.DateOfUpdate = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    if resultsCDF.Region    != CALCULATIONS_RegionName: ErrorMsg = "Save aborted: NetCDF file has already data about region " + resultsCDF.Region + " and you tried to save data about region " + CALCULATIONS_RegionName        
    if resultsCDF.OrbitFile != CALCULATIONS_OrbitFilesPath: ErrorMsg = "Save aborted: NetCDF file has already data about orbit " + resultsCDF.OrbitFile + " and you tried to save data about orbit " + CALCULATIONS_OrbitFilesPath
    if resultsCDF.DataPath  != CALCULATIONS_TIEGCMfolder: ErrorMsg = "Save aborted: NetCDF file has already data about TIEGCM file " + resultsCDF.DataPath  + "and you tried to save data about TIEGCM file " + CALCULATIONS_TIEGCMfolder        
    if len(DataFilename)>0 and resultsCDF.Progress > DataFilename: ErrorMsg = "Save aborted: NetCDF file contains data about file: " + resultsCDF.Progress + " which is later than " + DataFilename
    if len(ErrorMsg) > 0:
        print( ErrorMsg )
        resultsCDF.close()
        return
    resultsCDF.LastExecDurationSec = ConvertLeadingZerosToSpaces("{0:.0f}".format(CALCULATIONS_ExecutionDuration)).strip()
    # save data for each bin spearately 
    for B in Bins:
        # save data about the hits inside the bin
        if len(B.Time_values) > 0:
            resultsCDF.variables[B.ID+"_TimeValues"][:]      = resultsCDF.variables[B.ID+"_TimeValues"][:].tolist() + B.Time_values
            resultsCDF.variables[B.ID+"_JHValues"][:]        = resultsCDF.variables[B.ID+"_JHValues"][:].tolist() + B.JH_values        
            resultsCDF.variables[B.ID+"_MagLatValues"][:]    = resultsCDF.variables[B.ID+"_MagLatValues"][:].tolist() + B.MagLat_values
            resultsCDF.variables[B.ID+"_MLTValues"][:]       = resultsCDF.variables[B.ID+"_MLTValues"][:].tolist() + B.MLT_values
            resultsCDF.variables[B.ID+"_AltitudeValues"][:]  = resultsCDF.variables[B.ID+"_AltitudeValues"][:].tolist() + B.Altitude_values
            resultsCDF.variables[B.ID+"_LatValues"][:]       = resultsCDF.variables[B.ID+"_LatValues"][:].tolist() + B.Lat_values
            resultsCDF.variables[B.ID+"_KpValues"][:]        = resultsCDF.variables[B.ID+"_KpValues"][:].tolist() + B.Kp_values
            resultsCDF.variables[B.ID+"_EEXValues"][:]       = resultsCDF.variables[B.ID+"_EEXValues"][:].tolist() + B.EEX_values        
            resultsCDF.variables[B.ID+"_EEYValues"][:]       = resultsCDF.variables[B.ID+"_EEYValues"][:].tolist() + B.EEY_values
            resultsCDF.variables[B.ID+"_PedersenValues"][:]  = resultsCDF.variables[B.ID+"_PedersenValues"][:].tolist() + B.Pedersen_values
            resultsCDF.variables[B.ID+"_HallValues"][:]      = resultsCDF.variables[B.ID+"_HallValues"][:].tolist() + B.Hall_values
            resultsCDF.variables[B.ID+"_ConvectionHeatingValues"][:] = resultsCDF.variables[B.ID+"_ConvectionHeatingValues"][:].tolist() + B.ConvectionHeating_values
            resultsCDF.variables[B.ID+"_WindHeatingValues"][:] = resultsCDF.variables[B.ID+"_WindHeatingValues"][:].tolist() + B.WindHeating_values
    ## save data for all hits
    if len(all_Time_values) > 0:
        resultsCDF.variables["allTimeValues"][:]     = resultsCDF.variables["allTimeValues"][:].tolist() + all_Time_values
        resultsCDF.variables["allJHValues"][:]       = resultsCDF.variables["allJHValues"][:].tolist() + all_JH_values    
        resultsCDF.variables["allMagLatValues"][:]   = resultsCDF.variables["allMagLatValues"][:].tolist() + all_MagLat_values
        resultsCDF.variables["allMLTValues"][:]      = resultsCDF.variables["allMLTValues"][:].tolist() + all_MLT_values
        resultsCDF.variables["allAltitudeValues"][:] = resultsCDF.variables["allAltitudeValues"][:].tolist() + all_Altitude_values
        resultsCDF.variables["allLatValues"][:]      = resultsCDF.variables["allLatValues"][:].tolist() + all_Lat_values
        resultsCDF.variables["allKpValues"][:]       = resultsCDF.variables["allKpValues"][:].tolist() + all_Kp_values
        #resultsCDF.variables["allHittedBinIDs"][:]   = resultsCDF.variables["allHittedBinIDs"][:].tolist() + netCDF4.stringtochar(np.array(all_HittedBin_IDs[:], 'S8'))
        resultsCDF.variables["allEEXValues"][:]      = resultsCDF.variables["allEEXValues"][:].tolist() + all_EEX_values
        resultsCDF.variables["allEEYValues"][:]      = resultsCDF.variables["allEEYValues"][:].tolist() + all_EEY_values
        resultsCDF.variables["allPedersenValues"][:] = resultsCDF.variables["allPedersenValues"][:].tolist() + all_Pedersen_values
        resultsCDF.variables["allHallValues"][:]     = resultsCDF.variables["allHallValues"][:].tolist() + all_Hall_values
        resultsCDF.variables["allConvectionHeatingValues"][:] = resultsCDF.variables["allConvectionHeatingValues"][:].tolist() + all_ConvectionHeating_values
        resultsCDF.variables["allWindHeatingValues"][:] = resultsCDF.variables["allWindHeatingValues"][:].tolist() + all_WindHeating_values
    #
    resultsCDF.close()    
    



    


def LoadResults_CDF( filepath, loadBinValues=True, loadGlobalValues=True, loadTimeValues=True, loadMagLatValues=True, loadMLTvalues=True, loadAltValues=True, loadLatValues=True, loadKpValues=True ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilesPath, CALCULATIONS_TIEGCMfolder, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values, all_Kp_values, all_Time_values, all_EEX_values, all_EEY_values, all_Pedersen_values, all_Hall_values

    print( "Started Loading", filepath, datetime.now() )

    # make a list of all the files we are going to load
    All_ResultFilenames = list()
    if filepath[-1] == '/':
        All_ResultFilenames = sorted( glob.glob(filepath+"*.nc") )
    else:
        All_ResultFilenames.append( filepath )
    
    # load each file into memory
    for file_idx in range(0, len(All_ResultFilenames)):
        if file_idx % 10 == 0: print( "Now Loading", All_ResultFilenames[file_idx] )
        #if file_idx == 6: break
        resultsCDF = Dataset( All_ResultFilenames[file_idx], 'r' )
        #### load general information
        if file_idx == 0:
            try:
                print( "DateOfCreation:", resultsCDF.DateOfCreation, " LastExecDurationSec :", resultsCDF.LastExecDurationSec , "sec" )
                #print( "Title:", resultsCDF.Title, " Description:", resultsCDF.Description )
                print( "Region:", resultsCDF.Region )
                print( "OrbitFile:", resultsCDF.OrbitFile )
                print( "TIEGCM data path:", resultsCDF.DataPath, "\n" )
                #print( "Progress:", resultsCDF.Progress, "\n" )
            except:
                pass
            CALCULATIONS_Title = resultsCDF.Title
            CALCULATIONS_Description = resultsCDF.Description
            #CALCULATIONS_ExecutionDuration = resultsCDF.LastExecDurationSec
            CALCULATIONS_RegionName = resultsCDF.Region
            CALCULATIONS_OrbitFilesPath = resultsCDF.OrbitFile.split()
            CALCULATIONS_TIEGCMfolder = resultsCDF.DataPath
        #### load data for each bin
        if loadBinValues:
            for B in Bins:
                try:
                    if loadTimeValues and len(CALCULATIONS_OrbitFilesPath) > 0: concatLists( B.Time_values, list(resultsCDF.variables[ B.ID+"_TimeValues" ][:]) )
                    if loadMagLatValues: concatLists( B.MagLat_values, list(resultsCDF.variables[ B.ID+"_MagLatValues" ][:]) )
                    if loadMLTvalues: concatLists( B.MLT_values, list(resultsCDF.variables[ B.ID+"_MLTValues" ][:]) )
                    if loadAltValues: concatLists( B.Altitude_values, list(resultsCDF.variables[ B.ID+"_AltitudeValues" ][:]) )
                    try:
                        if loadLatValues: concatLists(B.Lat_values, list(resultsCDF.variables[ B.ID+"_LatValues" ][:]) )
                    except:
                        pass
                    if loadKpValues: concatLists(B.Kp_values, list(resultsCDF.variables[ B.ID+"_KpValues" ][:]) )
                    try:
                        if SELECTED_VARIABLE == "Ohmic":     concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_ConvectionHeatingValues" ][:]+resultsCDF.variables[ B.ID+"_WindHeatingValues" ][:]) )
                    except:
                        if SELECTED_VARIABLE == "Ohmic":     concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_ConvenctionHeatingValues" ][:]+resultsCDF.variables[ B.ID+"_WindHeatingValues" ][:]) )
                    if SELECTED_VARIABLE == "EEX_si":    concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_EEXValues" ][:])*1000 ) #if SELECTED_VARIABLE == "EEX_si":    B.EEX_values = list(resultsCDF.variables[ B.ID+"_EEXValues" ][:])
                    if SELECTED_VARIABLE == "EEY_si":    concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_EEYValues" ][:])*1000 ) #if SELECTED_VARIABLE == "EEY_si":    B.EEY_values = list(resultsCDF.variables[ B.ID+"_EEYValues" ][:])
                    if SELECTED_VARIABLE == "SIGMA_PED": concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_PedersenValues" ][:]) ) #if SELECTED_VARIABLE == "SIGMA_PED": B.Pedersen_values = list(resultsCDF.variables[ B.ID+"_PedersenValues" ][:])
                    if SELECTED_VARIABLE == "SIGMA_HAL": concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_HallValues" ][:]) ) #if SELECTED_VARIABLE == "SIGMA_HAL": B.Hall_values = list(resultsCDF.variables[ B.ID+"_HallValues" ][:])
                    try:
                        if SELECTED_VARIABLE == "Convection_heating": concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_ConvectionHeatingValues" ][:]) )
                    except:
                        if SELECTED_VARIABLE == "Convection_heating": concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_ConvenctionHeatingValues" ][:]) )
                    if SELECTED_VARIABLE == "Wind_heating": concatLists( B.JH_values, list(resultsCDF.variables[ B.ID+"_WindHeatingValues" ][:]) )
                except: # data about this region do not exist inside this netcdf file
                    continue
        #### load collective data about all bins
        if loadTimeValues and len(CALCULATIONS_OrbitFilesPath) > 0: concatLists( all_Time_values, list(resultsCDF.variables[ "allTimeValues" ][:]) )
        if loadMagLatValues:  concatLists( all_MagLat_values, list(resultsCDF.variables[ "allMagLatValues" ][:]) )
        if loadMLTvalues: concatLists( all_MLT_values, list(resultsCDF.variables[ "allMLTValues" ][:]) )
        if loadAltValues: concatLists( all_Altitude_values, list(resultsCDF.variables[ "allAltitudeValues" ][:]) )
        try:
            if loadLatValues: concatLists( all_Lat_values, list(resultsCDF.variables[ "allLatValues" ][:]) )
        except:
            pass
        if loadKpValues: concatLists( all_Kp_values, list(resultsCDF.variables[ "allKpValues" ][:]) )
        #if SELECTED_VARIABLE == "Ohmic":     concatLists( all_JH_values, list(resultsCDF.variables[ "allJHValues" ][:]) )
        
        try:
            if SELECTED_VARIABLE == "Ohmic":     concatLists( all_JH_values, list(resultsCDF.variables[ "allConvectionHeatingValues" ][:] + resultsCDF.variables[ "allWindHeatingValues" ][:]) )
        except:
            if SELECTED_VARIABLE == "Ohmic":     concatLists( all_JH_values, list(resultsCDF.variables[ "allConvenctionHeatingValues" ][:] + resultsCDF.variables[ "allWindHeatingValues" ][:]) )
        if SELECTED_VARIABLE == "EEX_si":    concatLists( all_JH_values, list(resultsCDF.variables[ "allEEXValues" ][:]*1000) )#if SELECTED_VARIABLE == "EEX_si":    all_EEX_values = list(resultsCDF.variables[ "allEEXValues" ][:])
        if SELECTED_VARIABLE == "EEY_si":    concatLists( all_JH_values, list(resultsCDF.variables[ "allEEYValues" ][:]*1000) )#if SELECTED_VARIABLE == "EEY_si":    all_EEY_values = list(resultsCDF.variables[ "allEEYValues" ][:])
        if SELECTED_VARIABLE == "SIGMA_PED": concatLists( all_JH_values, list(resultsCDF.variables[ "allPedersenValues" ][:]) )#if SELECTED_VARIABLE == "SIGMA_PED": all_Pedersen_values = list(resultsCDF.variables[ "allPedersenValues" ][:])
        if SELECTED_VARIABLE == "SIGMA_HAL": concatLists( all_JH_values, list(resultsCDF.variables[ "allHallValues" ][:]) )#if SELECTED_VARIABLE == "SIGMA_HAL": all_Hall_values = list(resultsCDF.variables[ "allHallValues" ][:])
        try:
            if SELECTED_VARIABLE == "Convection_heating": concatLists( all_JH_values, list(resultsCDF.variables[ "allConvectionHeatingValues" ][:]) )
        except:
            if SELECTED_VARIABLE == "Convection_heating": concatLists( all_JH_values, list(resultsCDF.variables[ "allConvenctionHeatingValues" ][:]) )
        if SELECTED_VARIABLE == "Wind_heating": concatLists( all_JH_values, list(resultsCDF.variables[ "allWindHeatingValues" ][:]) )
        #### close and go on
        resultsCDF.close()
    ########
    CalculateJouleHeatingPerBin()
    print( "Results loaded for", SELECTED_VARIABLE_longname, "    ", datetime.now(), "\n" )

    
def concatLists( a, b ):
    for item in b:
        a.append( item )    
    
    
    
def LoadResults( filename ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilesPath, CALCULATIONS_TIEGCMfolder, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    if len(filename) > 0:
        CALCULATIONS_ResultsFilename = ResultsFilename = filename
        with open(CALCULATIONS_ResultsFilename, 'r') as F:
            for line in F:
                if line[0:1] == '#': # this line contains a comment, print it as it is.
                    print ( line[1:len(line)-1] )
                    if line.startswith("# Title:"): CALCULATIONS_Title = line[8:].strip()
                    if line.startswith("# Description:"): CALCULATIONS_Description = line[14:].strip()
                    if line.startswith("# Region:"): CALCULATIONS_RegionName = line[9:].strip()
                    if line.startswith("# Orbit Filename:"): CALCULATIONS_OrbitFilesPath = line[17:].strip()
                    if line.startswith("# DataPath"): CALCULATIONS_TIEGCMfolder = line[10:].strip()
                elif line.startswith( "All JH values" ) :
                    all_JH_values = line[line.find(":")+1:].split(',')
                    all_JH_values = [float(i) for i in all_JH_values]
                elif line.startswith( "All MagLat values" ) :
                    all_MagLat_values = line[line.find(":")+1:].split(',')
                    all_MagLat_values = [float(i) for i in all_MagLat_values]
                elif line.startswith( "All MLT values" ) :
                    all_MLT_values = line[line.find(":")+1:].split(',')
                    all_MLT_values = [float(i) for i in all_MLT_values]
                elif line.startswith( "All Altitude values" ) :
                    all_Altitude_values = line[line.find(":")+1:].split(',')
                    all_Altitude_values = [float(i) for i in all_Altitude_values]                    
                elif line.startswith( "BIN" ):
                    head_of_line = line[0:20]
                    bin_id = head_of_line[ 4: head_of_line.find(':') ]
                    B = getBinByItsID( bin_id )
                    data_str = line[line.find("=")+1:-1].strip()
                    if len( data_str ) > 0:
                        values = data_str.split(',')
                        values = [float(i) for i in values]
                    else:
                        values = list()
                    if   "MagLat"   in head_of_line: 
                        B.MagLat_values   = values
                    elif "MLT"      in head_of_line: 
                        B.MLT_values      = values
                    elif "Altitude" in head_of_line: 
                        B.Altitude_values = values    
                else: # this line contains bin info, print it and store them in the correct bin.
                    s = line[:220]
                    if s[-1] == '\n': s = s[:-1]
                    print ( s )
                    aBinID = line[:line.find(":")].strip()
                    ##
                    str_JH = line[line.find("JH_values=")+10:-1]
                    if len( str_JH.strip() ) > 0:
                        aBinJH_values = str_JH.split(',')
                        aBinJH_values = [float(i) for i in aBinJH_values]
                        for B in Bins:
                            if B.ID == aBinID:
                                B.JH_values = aBinJH_values
                                break
        F.close()
        CalculateJouleHeatingPerBin()
    
    
    
    
    
    
@cuda.jit
def Match(MAGLATs, MLTs, ALTs, KPs, JHs,   MagLat_min, MagLat_max, MLT_min,MLT_max, Altitude_min,Altitude_max, Kp_min,Kp_max,   out):
    length_time =  60
    length_lev  =  57
    length_lat  =  72
    length_lon  = 144
    num_of_blocks  = cuda.gridDim.x   # number of blocks in the grid
    num_of_threads = cuda.blockDim.x  # number of threads per block
    BlockIDX  = cuda.blockIdx.x  # this is the unique block ID within the 1D grid
    ThreadIDX = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    #start = tx + ty * block_size
    #stride = block_size * grid_size
    BlockStep  = int(length_lat/num_of_blocks)+1
    ThreadStep = int(length_lon/num_of_threads)+1
    for idx_lat in range( BlockIDX*BlockStep,  BlockIDX*BlockStep + BlockStep):
        if idx_lat >= length_lat: continue
        for idx_lon in range( ThreadIDX*ThreadStep,  ThreadIDX*ThreadStep + ThreadStep):
            if idx_lon >= length_lon: continue
            for idx_lev in range(0, length_lev):
                if idx_lev >= length_lev: continue
                for idx_time in range(0, length_time):
                    if idx_time >= length_time: continue
                    if MAGLATs[idx_time, idx_lev, idx_lat, idx_lon]      >= MagLat_min       and MAGLATs[idx_time, idx_lev, idx_lat, idx_lon] <= MagLat_max:
                        if ALTs[idx_time, idx_lev, idx_lat, idx_lon] >= Altitude_min     and ALTs[idx_time, idx_lev, idx_lat, idx_lon]    <= Altitude_max:
                            if KPs[idx_time] >= Kp_min and KPs[idx_time] <= Kp_max:
                                is_MLT_in_range = False
                                if MLT_min <= MLT_max: # example: from 13 to 18 hour
                                    if MLTs[idx_time, idx_lev, idx_lat, idx_lon] >= MLT_min and MLTs[idx_time, idx_lev, idx_lat, idx_lon] <= MLT_max: is_MLT_in_range = True
                                else: # example: from 22 to 3 hour
                                    if MLTs[idx_time, idx_lev, idx_lat, idx_lon] > MLT_min or MLTs[idx_time, idx_lev, idx_lat, idx_lon] <= MLT_max: is_MLT_in_range = True
                                if is_MLT_in_range: out[idx_time, idx_lev, idx_lat, idx_lon] = JHs[idx_time, idx_lev, idx_lat, idx_lon]

def AssignJouleHeatingValuesToBins_CUDA( DataFilesPath ): # MagLat MagLoacalTime Kp Alt
    startSecs = time.time()        
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Lat_min     =  1000
    Lat_max     = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    Matches = 0
    
    AllDataFiles = sorted( glob.glob( DataFilesPath + "*/*.nc", recursive=True ) )
    for currentDataFile in AllDataFiles:
        print( "Reading", currentDataFile )
        try:
            CDFroot = Dataset( currentDataFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        try:
            FileStartTimeStamp = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"    FileStartTimeStamp = calendar.timegm( datetime.strptime( "minutes since 2015-1-1 0:0:0"[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() )
        except:
            print ( "WRONG CONTENTS:", currentDataFile )
            continue            
        length_time = CDFroot.variables['Ohmic'].shape[0]
        length_lev  = CDFroot.variables['Ohmic'].shape[1]
        length_lat  = CDFroot.variables['Ohmic'].shape[2]
        length_lon  = CDFroot.variables['Ohmic'].shape[3]
        
        # Load or calculate all basic values from the netcdf file
        print ("Loading file into memory")
        TIMEs   = CDFroot.variables['time'][:] # minutes since the start time
        LATs    = CDFroot.variables['lat'][:] 
        MAGLATs = CDFroot.variables['mlat_qdf'][:, :, :, :] 
        MLTs    = CDFroot.variables['mlt_qdf'][:, :, :, :]         
        ALTs    = CDFroot.variables['ZGMID'][:, :, :, :] / 100000 # it is stored in cm inside the file
        KPs     = CDFroot.variables['Kp'][:]
        JHs     = CDFroot.variables['Ohmic'][:, :, :, :]
        EEXs    = CDFroot.variables['EEX_si'][:, :, :, :] 
        EEYs    = CDFroot.variables['EEY_si'][:, :, :, :] 
        PEDs    = CDFroot.variables['SIGMA_PED'][:, :, :, :] 
        HALs    = CDFroot.variables['SIGMA_HAL'][:, :, :, :] 

        
        print( "Invoking CUDA" )
        MAGLATs = np.array( MAGLATs, dtype="float32" )
        MLTs = np.array( MLTs, dtype="float32" )        
        ALTs = np.array( ALTs, dtype="float32" )
        KPs = np.array( KPs, dtype="float32" )
        JHs = np.array( JHs, dtype="float32" )
        out = np.zeros(  [length_time, length_lev, length_lat, length_lon]  )
        blocks_per_grid = (16)
        threads_per_block = (32)
        Match[blocks_per_grid, threads_per_block](MAGLATs, MLTs, ALTs, KPs, JHs,   MagLat_min, MagLat_max, MLT_min,MLT_max, Altitude_min,Altitude_max, Kp_min,Kp_max,   out)
        #device_out.copy_to_host()
        print( "Positions in Bins:", np.count_nonzero(out), "/" , out.size)
        
        for idx_lat in range(0, length_lat):
            for idx_lon in range(0, length_lon):
                for idx_lev in range(0, length_lev):
                    for idx_time in range(0, length_time):
                        if out[idx_time, idx_lev, idx_lat ,idx_lon] != 0:
                            current_MLT      = MLTs[idx_time, idx_lev, idx_lat ,idx_lon]
                            current_MagLat   = MAGLATs[idx_time, idx_lev, idx_lat ,idx_lon]
                            current_Altitude = ALTs[idx_time, idx_lev, idx_lat ,idx_lon]
                            current_Kp       = KPs[idx_time]
                            current_Lat      = LATs[ idx_lat ]
                            matchedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp, current_Lat )
                            if matchedBin is not None:
                                current_timestamp = FileStartTimeStamp + TIMEs[idx_time]*120*60
                                current_JH = JHs[idx_time, idx_lev, idx_lat ,idx_lon] #CDFroot.variables['Joule Heating'][idx_time, idx_lev, idx_lat, idx_lon]
                                matchedBin.JH_values.append( current_JH )
                                matchedBin.MagLat_values.append( current_MagLat )
                                matchedBin.MLT_values.append( current_MLT )
                                matchedBin.Altitude_values.append( current_Altitude )
                                matchedBin.Kp_values.append( current_Kp )
                                matchedBin.Time_values.append( current_timestamp )    
                                matchedBin.EEX_values.append( EEXs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.EEY_values.append( EEYs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.Pedersen_values.append( PEDs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.Hall_values.append( HALs[ idx_time, idx_lev, idx_lat, idx_lon ] )                                 
                                all_JH_values.append( current_JH )
                                all_MagLat_values.append( current_MagLat )
                                all_MLT_values.append( current_MLT )
                                all_Altitude_values.append( current_Altitude )
                                all_Kp_values.append( current_Kp )
                                all_Time_values.append( current_timestamp )                                
                                all_HittedBin_IDs.append( matchedBin.ID )
                                all_EEX_values.append( EEXs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_EEY_values.append( EEYs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_Pedersen_values.append( PEDs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_Hall_values.append( HALs[ idx_time, idx_lev, idx_lat, idx_lon ] )                                
                                Matches += 1
        
        CDFroot.close()
        print( Matches, " matches so far." )
    finishSecs = time.time()
    print( finishSecs-startSecs, " sec")    
    print( Matches, "points have been assigned into bins" )

            
#######################            #######################            #######################            
                    #######################            #######################            
def AssignJouleHeatingValuesToBins( DataFilesPath ):
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values, all_Lat_values, all_Kp_values, all_Time_values, all_HittedBin_IDs  , all_EEX_values, all_EEY_values, all_Pedersen_values, all_Hall_values
    startSecs = time.time()
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Lat_min     =  1000
    Lat_max     = -1000    
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Lat_min < Lat_min: Lat_min = B.Lat_min 
        if B.Lat_max > Lat_max: Lat_max = B.Lat_max                        
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    Matches = 0

    # read the results file and find out until which TIEGCM file's data have been processed into it
    ResultsFilename = DaedalusGlobals.CoverageResults_Files_Path + BinGroups_Dropdown.value + "." + CALCULATIONS_TIEGCMfolder[CALCULATIONS_TIEGCMfolder[:-1].rfind('/')+1:-1] + ".ValuesPerBinResults.nc"
    if path.exists( ResultsFilename ):
        resultsCDF = Dataset( ResultsFilename, 'r' )
        Progress = resultsCDF.Progress.strip()
        resultsCDF.close()
        if len(Progress) == 0:
            skip = False
        else:
            skip = True
    else:
        CreateResults_CDF( ResultsFilename )
        skip = False
    
    AllDataFiles = sorted( glob.glob( DataFilesPath + "*/*.nc", recursive=True ) )
    for currentDataFile in AllDataFiles:
        # skip the files which have already been processed inti the results file
        if skip == True: 
            if currentDataFile.strip()==Progress: skip = False
            print( "\nSkiping Data file:", currentDataFile)
            continue # <<< this file has been parsed, go on
        # reset state
        all_JH_values       = list()
        all_MagLat_values   = list() 
        all_MLT_values      = list() 
        all_Altitude_values = list() 
        all_Lat_values      = list()
        all_Kp_values       = list() 
        all_Time_values     = list()
        all_HittedBin_IDs   = list()
        all_EEX_values      = list()
        all_EEY_values      = list()
        all_Pedersen_values = list()
        all_Hall_values     = list()
        for B in Bins:
            B.reset()
            
        # parse TIEGCM file
        print( "Reading", currentDataFile )
        try:
            CDFroot = Dataset( currentDataFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        try:
            FileStartTimeStamp = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"
        except:
            print ( "WRONG CONTENTS:", currentDataFile )
            continue                    
        length_time = CDFroot.variables['Ohmic'].shape[0]
        length_lev  = CDFroot.variables['Ohmic'].shape[1]
        length_lat  = CDFroot.variables['Ohmic'].shape[2]
        length_lon  = CDFroot.variables['Ohmic'].shape[3]
        # Load or calculate all basic values from the netcdf file
        print ("Loading file into memory")
        TIMEs   = CDFroot.variables['time'][:] # minutes since the start time
        LATs    = CDFroot.variables['lat'][:] 
        ALTs    = CDFroot.variables['ZGMID'][:, :, :, :] / 100000 # it is stored in cm inside the file
        JHs     = CDFroot.variables['Ohmic'][:, :, :, :]
        KPs     = CDFroot.variables['Kp'][:]
        MAGLATs = CDFroot.variables['mlat_qdf'][:, :, :, :] 
        MLTs    = CDFroot.variables['mlt_qdf'][:, :, :, :] 
        EEXs    = CDFroot.variables['EEX_si'][:, :, :, :] 
        EEYs    = CDFroot.variables['EEY_si'][:, :, :, :] 
        PEDs    = CDFroot.variables['SIGMA_PED'][:, :, :, :] 
        HALs    = CDFroot.variables['SIGMA_HAL'][:, :, :, :] 

        step = 1
        for idx_lat in range(0, length_lat, step):
            if idx_lat%30==0: print("Calculating Lat",  idx_lat)
            for idx_lon in range(0, length_lon, step):
                for idx_lev in range(0, length_lev, step):
                    for idx_time in range(0, length_time, step):
                        in_Altitude_range = in_MagLat_range = in_MLT_range = in_Kp_range = in_Lat_range = False
                        
                        current_Altitude = ALTs[idx_time, idx_lev, idx_lat, idx_lon]
                        if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max:
                                in_Altitude_range = True
                        
                        if in_Altitude_range:
                            #current_Latitude  = LATs[ idx_lat ]    current_Longitude = LONs[ idx_lon ]     geodetic_Latitude = Conversions.geo_lat2geod_lat( current_Latitude )    TimeObj = datetime.fromtimestamp( FileStartTimeStamp + 60*TIMEs[idx_time], tz=timezone.utc )       current_MagLat, current_MagLon, current_MLT = Conversions.getMagneticProperties( TimeObj, geodetic_Latitude, current_Longitude, current_Altitude )
                            current_MagLat = MAGLATs[ idx_time, idx_lev, idx_lat, idx_lon ]
                            if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max:
                                in_MagLat_range = True
                                
                        if in_MagLat_range:
                            current_MLT = MLTs[ idx_time, idx_lev, idx_lat, idx_lon ]
                            if in_MagLat_range:
                                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )
                        
                        if in_MLT_range:
                            current_Kp = KPs[idx_time]
                            if current_Kp >= Kp_min and current_Kp <= Kp_max:
                                in_Kp_range = True   
                                
                        if in_Kp_range: 
                            current_Lat = LATs[ idx_lat ]
                            if current_Lat >= Lat_min and current_Lat <= Lat_max: in_Lat_range = True   
                        ##
                        if in_Lat_range:
                            matchedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp, current_Lat )
                            if matchedBin is not None:
                                current_time = int( FileStartTimeStamp + TIMEs[idx_time]*120*60 )
                                current_JH = JHs[idx_time, idx_lev, idx_lat ,idx_lon] #CDFroot.variables['Joule Heating'][idx_time, idx_lev, idx_lat, idx_lon]
                                matchedBin.JH_values.append( current_JH )
                                matchedBin.MagLat_values.append( current_MagLat )
                                matchedBin.MLT_values.append( current_MLT )
                                matchedBin.Altitude_values.append( current_Altitude )
                                matchedBin.Kp_values.append( current_Kp )
                                matchedBin.Time_values.append( current_time )
                                matchedBin.EEX_values.append( EEXs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.EEY_values.append( EEYs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.Pedersen_values.append( PEDs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.Hall_values.append( HALs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                all_JH_values.append( current_JH )
                                all_MagLat_values.append( current_MagLat )
                                all_MLT_values.append( current_MLT )
                                all_Altitude_values.append( current_Altitude )
                                all_Kp_values.append( current_Kp )
                                all_Time_values.append( current_time )
                                all_HittedBin_IDs.append( matchedBin.ID )
                                all_EEX_values.append( EEXs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_EEY_values.append( EEYs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_Pedersen_values.append( PEDs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_Hall_values.append( HALs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                Matches += 1
                            else:
                                print( "PARADOX at:", current_MLT, current_MagLat, current_Altitude, current_Kp, " :: ", idx_time, idx_lev, idx_lat, idx_lon )
                #break
            #break
        CDFroot.close()
        SaveResults_CDF( ResultsFilename, currentDataFile )
    finishSecs = time.time()
    print( finishSecs-startSecs, " sec")    
    print( Matches, "points have been assigned into bins" )
    return ResultsFilename
    
    
    
    
'''
TIEGCM_filesPath: the folder which has all TIEGCM netcdf files describing Earth's enviroment during the orbit's duration
OrbitFilesPath: the folder which has all netCDF files which contain all the positions of the satellite 
'''    
def AssignJouleHeatingValuesToBins_AlongOrbit( TIEGCM_filesPath, Orbit_filesPath ): # MagLat MagLoacalTime Kp Alt 
    # initialize
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Lat_min     =  1000
    Lat_max     = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Lat_min < Lat_min: Lat_min = B.Lat_min 
        if B.Lat_max > Lat_max: Lat_max = B.Lat_max                                
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max                    
    # miscellaneous
    currentfilenumber = -1        
    Matches = 0
    Errors  = 0
    # information about the TIEGCM files
    TIEGCMfilenamePrefix  = "tiegcm2.0_res2.5_3years_sech_" 
    TIEGCMfilenamePostfix = "_JH_QD_AllVars"

    # read orbit file
    current_timestamp_offset = 0 # increases after each satellite position is parsed
    AllOrbitFiles = sorted( glob.glob( Orbit_filesPath + "*Batch*.nc" ) )
    for currentOrbitFile in AllOrbitFiles:
        print( "\nReading Orbit file:", currentOrbitFile )
        try:
            Orbit_CDF = Dataset( currentOrbitFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        # Load data from the netCDF file
        ORBIT_Times      = Orbit_CDF.variables['time'][:]
        ORBIT_MagLats    = Orbit_CDF.variables['DaedalusMagneticLatitude'][:]
        ORBIT_MLTs       = Orbit_CDF.variables['DaedalusMLT'][:]
        ORBIT_Altitudes  = Orbit_CDF.variables['ZGMID'][:] / 100000
        ORBIT_Lats       = Orbit_CDF.variables['lat'][:]
        ORBIT_Ohmic      = Orbit_CDF.variables['Ohmic'][:]
        try:
            ORBIT_ConvH      = Orbit_CDF.variables['Convection_heating'][:]
        except:
            ORBIT_ConvH      = Orbit_CDF.variables['Convenction_heating'][:]
        ORBIT_WindH      = Orbit_CDF.variables['Wind_heating'][:]
        try: 
            ORBIT_EEX    = Orbit_CDF.variables['EEX_si'][:] 
        except: 
            ORBIT_EEX    = list()
        try: 
            ORBIT_EEY    = Orbit_CDF.variables['EEY_si'][:] 
        except: 
            ORBIT_EEY    = list()  
        try: 
            ORBIT_Pedersen = Orbit_CDF.variables['SIGMA_PED'][:] 
        except: 
            ORBIT_Pedersen = list()            
        try: 
            ORBIT_Hall    = Orbit_CDF.variables['SIGMA_HAL'][:] 
        except: 
            ORBIT_Hall    = list()
            
        orbit_start_datetime  = datetime.strptime(Orbit_CDF.variables['time'].UNITS[14:], '%d %b %Y %H:%M:%S.%f')
        orbit_start_timestamp = calendar.timegm(orbit_start_datetime.utctimetuple())
        orbit_timestamp_step = ORBIT_Times[1] - ORBIT_Times[0]
        print( "orbit_timestamp_step =", orbit_timestamp_step )
        num_of_positions =  len(ORBIT_Times)
        # read the satellite positions and try to fill the bins
        for idx in range(0, num_of_positions): # for each satellite position
            if idx % 200000 == 0: print ("Checking sat position No", idx, "of", num_of_positions)
            in_Altitude_range = in_MagLat_range = in_MLT_range = in_Lat_range = in_Kp_range = False
                      
            # check if this position lies inside some bin
            current_Altitude = ORBIT_Altitudes[ idx ]
            if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max: in_Altitude_range = True
            #
            if in_Altitude_range:
                current_MagLat = ORBIT_MagLats[ idx ]
                if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max: in_MagLat_range = True
            #
            if in_MagLat_range:
                current_MLT = ORBIT_MLTs[ idx ]
                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )
                
            # 
            if in_MLT_range: 
                current_Lat = ORBIT_Lats[ idx ]
                if current_Lat >= Lat_min and current_Lat <= Lat_max: in_Lat_range = True
                    
            if in_Lat_range==False:
                current_MagLat = ORBIT_MagLats[ idx ]
                current_MLT = ORBIT_MLTs[ idx ]
                current_Lat = ORBIT_Lats[ idx ]
                if idx % 200000 == 0: print( "ALT:",current_Altitude, "MAGLAT:", current_MagLat, "MLT:", current_MLT, "LAT:",current_Lat )

            # The position is probably inside a bin (only kp remains to be checked). 
            # Open the corresponding TIEGCM file to read the kp and if position is in bin then calculate JH
            if in_Lat_range:
                current_timestamp = orbit_start_timestamp + current_timestamp_offset
                current_datetime  = datetime.utcfromtimestamp( current_timestamp )
                
                # Locate the corresponding TIEGCM file and timestep inside the file
                # one TIEGCM file contains 60 timesteps, 1 per 120min. The file's duration is 5 days. Each year consists of 74 files (the last file is smaller)
                start_of_current_year_datetime  = datetime.strptime("01 Jan " + str(current_datetime.year) + " 00:00:00", '%d %b %Y %H:%M:%S')
                start_of_current_year_timestamp = calendar.timegm(start_of_current_year_datetime.utctimetuple())
                newfilenumber = int(  ( (current_timestamp - start_of_current_year_timestamp)/(60*120) ) / 60  ) 
                tmp = (current_timestamp - start_of_current_year_timestamp)/(60*120) - newfilenumber*60 
                timestep_number = int( tmp )
                if tmp - float(timestep_number) > 0.5: timestep_number += 1 # select the nearest neighbor
                if  ( current_timestamp==start_of_current_year_timestamp  or  (current_timestamp - start_of_current_year_timestamp)/(60*120) ) % 60  !=  0: newfilenumber += 1 # file numbers start from 1
                if current_datetime.year == 2016: newfilenumber += 74
                if current_datetime.year == 2017: newfilenumber += 148
                
                # open the TIEGCM file if necessary
                if currentfilenumber < 0   or   currentfilenumber != newfilenumber:
                    if currentfilenumber >= 0: tiegcm_CDF.close()
                    TIEGCMfilename = TIEGCM_filesPath + "TIEGCM_" + str(current_datetime.year) + "/" + TIEGCMfilenamePrefix + "{:03.0f}".format(newfilenumber) + TIEGCMfilenamePostfix + ".nc"
                    currentfilenumber = newfilenumber
                    print(  "Opening TIEGCMfile:", TIEGCMfilename)
                    try:
                        tiegcm_CDF = Dataset( TIEGCMfilename, 'r' )
                    except:
                        print ( "FILE NOT FOUND OR WRONG FORMAT:", TIEGCMfilename )
                        continue
                        
                # read Kp from the tiegcm file
                try:
                    current_Kp = tiegcm_CDF.variables['Kp'][timestep_number]
                except:
                    #print("%%%%%%%%%%%%%%%%%%%%%")
                    #print(len(tiegcm_CDF.variables['Kp']), timestep_number)
                    #print( current_datetime, current_timestamp, start_of_current_year_timestamp )
                    #print(TIEGCMfilename)
                    #print("%%%%%%%%%%%%%%%%%%%%%")
                    current_Kp = tiegcm_CDF.variables['Kp'][timestep_number-1]
                    
                if current_Kp >= Kp_min and current_Kp <= Kp_max:
                    in_Kp_range = True 
                    
                # if the satellite position matches a bin then mark it as a hit and remember the JH values 
                if in_MagLat_range and in_MLT_range and in_Altitude_range and in_Kp_range:
                    matchedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp, current_Lat )
                    if matchedBin is not None:
                        # for this position locate the neighbor latitudes at the TIEGCM file. 
                        #lat1_idx, lat2_idx, lat1_val, lat2_val = findNeighborValues( TIEGCM_Lats, current_GeogLat )
                        # for this position locate the neighbor longitudes at the TIEGCM file.
                        #lon1_idx, lon2_idx, lon1_val, lon2_val = findNeighborValues( TIEGCM_Lons, current_Lon )
                        # for this position locate the neighbor Altitudes at the TIEGCM file. 
                        #lev1_idx, lev2_idx, lev1_val, lev2_val = findNeighborValues( CDFroot.variables['ZGMID'][time_idx, :, lat_idx, lon_idx], current_Altitude )
                        current_JH = ORBIT_Ohmic[ idx ]
                        if math.isnan(current_JH): 
                            print( "Found JH equal with None at file ", TIEGCMfilename, "timestep=", timestep_number, "altitude=", ORBIT_Altitudes[idx] )
                            Errors += 1
                            if Errors > 20:
                                print("Too many errors. Aborting")
                                return
                            else:
                                continue
                        # save 
                        matchedBin.JH_values.append( current_JH )
                        matchedBin.MagLat_values.append( current_MagLat )
                        matchedBin.MLT_values.append( current_MLT )
                        matchedBin.Altitude_values.append( current_Altitude )
                        matchedBin.Lat_values.append( current_Lat )
                        matchedBin.Kp_values.append( current_Kp )
                        matchedBin.Time_values.append( current_timestamp )
                        matchedBin.EEX_values.append( ORBIT_EEX[ idx ] ) 
                        matchedBin.EEY_values.append( ORBIT_EEY[ idx ] ) 
                        matchedBin.Pedersen_values.append( ORBIT_Pedersen[ idx ] ) 
                        matchedBin.ConvectionHeating_values.append( ORBIT_ConvH[ idx ] )
                        matchedBin.WindHeating_values.append( ORBIT_WindH[ idx ] )
                        if len(ORBIT_Hall) > 0: 
                            matchedBin.Hall_values.append( ORBIT_Hall[ idx ] ) 
                        all_JH_values.append( current_JH )
                        all_MagLat_values.append( current_MagLat )
                        all_MLT_values.append( current_MLT )
                        all_Altitude_values.append( current_Altitude )
                        all_Lat_values.append( current_Lat )
                        all_Kp_values.append( current_Kp )
                        all_Time_values.append( current_timestamp )
                        all_HittedBin_IDs.append( matchedBin.ID )
                        all_EEX_values.append( ORBIT_EEX[ idx ] )
                        all_EEY_values.append( ORBIT_EEY[ idx ] )
                        all_Pedersen_values.append( ORBIT_Pedersen[ idx ] )
                        all_ConvectionHeating_values.append( ORBIT_ConvH[ idx ] )
                        all_WindHeating_values.append( ORBIT_WindH[ idx ] )
                        if len(ORBIT_Hall) > 0: all_Hall_values.append( ORBIT_Hall[ idx ] )
                        Matches += 1
                    else:
                        print( "PARADOX at:", current_MLT, current_MagLat, current_Altitude, current_Kp, " :: ", time_idx, lev_idx, lat_idx, lon_idx )
            current_timestamp_offset += orbit_timestamp_step
    # clean up
    print( Matches, "satellite positions where matched inside bins." )
    try:
        CDFroot.close()
    except:
        print (".")
    
    
    
'''
TIEGCM_filesPath: the folder which has all TIEGCM netcdf files describing Earth's enviroment during the orbit's duration
OrbitFilename: csv file containing all the positions of the satellite 
'''    
def AssignJouleHeatingValuesToBins_AlongOrbit_forCSVorbit( TIEGCM_filesPath, OrbitFilename ): # MagLat MagLoacalTime Kp Alt 
    # initialize
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    # information about the TIEGCM files
    TIEGCMfilenamePrefix  = "tiegcm2.0_res2.5_3years_sech_" 
    TIEGCMfilenamePostfix = "_JH_QD_AllVars"
    CDFroot = Dataset( TIEGCM_filesPath + TIEGCMfilenamePrefix + "002" + TIEGCMfilenamePostfix + ".nc", 'r' )  # open a tiegcm file
    TIEGCM_StartTimeStamp  = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"
    #TIEGCM_StartTimeStamp  = calendar.timegm( datetime.strptime( "minutes since 2015-1-1 0:0:0"[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) 
    TIEGCM_TimeStep_sec    = (CDFroot.variables['time'][1] - CDFroot.variables['time'][0]) * 60 # every how many seconds a measurement is stored in the file
    TIEGCM_NumOfTimeSteps  = len( CDFroot.variables['time'][:] ) # the number of timesteps stored in the file
    TIEGCM_Lats = CDFroot.variables['lat'][:]
    TIEGCM_Lons = CDFroot.variables['lon'][:]
    CDFroot.close()
    # miscellaneous
    currentfilenumber = -1
    Matches = 0
    time_idx = lat_idx = lon_idx = lev_idx = -1
    print( "TIEGCM UNIVERSE:")
    print( "    Start Time =", "(UTC:"+str(TIEGCM_StartTimeStamp)+")", datetime.fromtimestamp(TIEGCM_StartTimeStamp) )
    print( "    Time-step  =", str(TIEGCM_TimeStep_sec)+"sec" + " #steps/file =", TIEGCM_NumOfTimeSteps, " Duration/file =", str(TIEGCM_NumOfTimeSteps*TIEGCM_TimeStep_sec/(60*60))+"hours", "\n" )
    
    # read orbit file
    with open( OrbitFilename ) as CSVfile:        
        CSVreader = csv.reader( CSVfile )
        # locate the column numnbers of interest inside the csv file
        CSVheader = next( CSVreader )
        Time_col     = CSVheader.index( "Epoch(UTCG)" ) #CSVheader.index( "Daedalus.EpochText" )
        Lat_col      = CSVheader.index( "Lat_GEOD(deg)" ) #CSVheader.index( "Daedalus.Latitude" )
        Lon_col      = CSVheader.index( "Lon_GEOD(deg)" ) #CSVheader.index( "Daedalus.Longitude" )
        Altitude_col = CSVheader.index( "Height_WGS84 (km)" ) #CSVheader.index( "Daedalus.Height" )
        MagLat_col   = CSVheader.index( "Daedalus.Magnetic Latitude" )
        MLT_col      = CSVheader.index( "Daedalus.MLT" )
        # read the satellite positions and try to fill the bins
        line_count = 0
        for row in CSVreader: # for each satellite position
            line_count += 1
            if line_count % 200000 == 0: print ("Checking sat position No", line_count, "of", row[Time_col])
            current_GeodLat = float( row[Lat_col] )
            current_GeogLat = float( row[Lat_col] ) # TODO: read correct column from orbit file
            current_Lon = float( row[Lon_col] )
            current_Altitude = float( row[Altitude_col] )
            in_Altitude_range = in_MagLat_range = in_MLT_range = in_Kp_range = False
                      
            # check if this position lies inside some bin
            current_Altitude = float( row[Altitude_col] )
            if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max:
                in_Altitude_range = True
            #
            if in_Altitude_range:
                current_MagLat = float( row[MagLat_col] )
                if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max:
                    in_MagLat_range = True
            #
            if in_MagLat_range:
                current_MLT = float( row[MLT_col] )
                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )

            # The position is probably inside a bin (only kp remains to be checked). 
            # Open the corresponding TIEGCM file to read the kp and if position is in bin then calculate JH
            if in_MLT_range:
                current_time = parseDaedalusDate( row[Time_col] )
                current_timestamp = calendar.timegm(current_time.utctimetuple())
                if current_time == None:
                    print( "ERROR during coverage calculation while reading", OrbitFilename, ": Wrong time format:", row[Time_col], "at line", line_count )
                    return # <<<<
                # TODO remove after testing:
                if current_time.year > 2024:  current_time = current_time - relativedelta(years=13)
                # open the correct TIEGCM file according to time
                current_timestep_number = (current_timestamp - TIEGCM_StartTimeStamp) / TIEGCM_TimeStep_sec
                newfilenumber = int(( current_timestep_number ) / TIEGCM_NumOfTimeSteps) + 1
                
                #print( "ZAZA", "orbit t=", current_time , "timestep_num=", current_timestep_number, "  filenum=", newfilenumber )
                if newfilenumber<=1 or newfilenumber > 73: continue # TODO del this line
                    
                if currentfilenumber < 0   or   currentfilenumber != newfilenumber:
                    if currentfilenumber >= 0: CDFroot.close()
                    TIEGCMfilename = TIEGCM_filesPath + TIEGCMfilenamePrefix + "{:03.0f}".format(newfilenumber) + TIEGCMfilenamePostfix + ".nc"
                    currentfilenumber = newfilenumber
                    print(  "Opening TIEGCMfile:", TIEGCMfilename)
                    try:
                        CDFroot = Dataset( TIEGCMfilename, 'r' )
                    except:
                        print ( "FILE NOT FOUND OR WRONG FORMAT:", TIEGCMfilename )
                        continue
                # calculate the time-step inside the TIEGCM which corresponds to the satellite time 
                time_idx = int(  current_timestep_number - (newfilenumber-1)*TIEGCM_NumOfTimeSteps  )
                # read Kp from the tiegcm file
                
                current_Kp = CDFroot.variables['Kp'][time_idx]
                if current_Kp >= Kp_min and current_Kp <= Kp_max:
                    in_Kp_range = True 
                # if the satellite position matches a bin then mark it as a hit and remember the JH values 
                if in_MagLat_range and in_MLT_range and in_Altitude_range and in_Kp_range:
                    matchedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp )
                    if matchedBin is not None:
                        # for this position locate the neighbor latitudes at the TIEGCM file. 
                        lat1_idx, lat2_idx, lat1_val, lat2_val = findNeighborValues( TIEGCM_Lats, current_GeogLat )
                        # for this position locate the neighbor longitudes at the TIEGCM file.
                        lon1_idx, lon2_idx, lon1_val, lon2_val = findNeighborValues( TIEGCM_Lons, current_Lon )
                        # for this position locate the neighbor Altitudes at the TIEGCM file. 
                        lev1_idx, lev2_idx, lev1_val, lev2_val = findNeighborValues( CDFroot.variables['ZGMID'][time_idx, :, lat_idx, lon_idx], current_Altitude )
                        # TODO: TRILINEAR INTERPOLATION
                        current_JH  = CDFroot.variables['Ohmic'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        current_EEX = CDFroot.variables['EEX_si'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        current_EEY = CDFroot.variables['EEY_si'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        current_PED = CDFroot.variables['SIGMA_PED'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        current_HAL = CDFroot.variables['SIGMA_HAL'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        # save 
                        matchedBin.JH_values.append( current_JH )
                        matchedBin.MagLat_values.append( current_MagLat )
                        matchedBin.MLT_values.append( current_MLT )
                        matchedBin.Altitude_values.append( current_Altitude )
                        matchedBin.Kp_values.append( current_Kp )
                        matchedBin.Time_values.append( current_timestamp )
                        matchedBin.EEX_values.append( current_EEX ) 
                        matchedBin.EEY_values.append( current_EEY ) 
                        matchedBin.Pedersen_values.append( current_PED ) 
                        matchedBin.Hall_values.append( current_HAL )                         
                        all_JH_values.append( current_JH )
                        all_MagLat_values.append( current_MagLat )
                        all_MLT_values.append( current_MLT )
                        all_Altitude_values.append( current_Altitude )
                        all_Kp_values.append( current_Kp )
                        all_Time_values.append( current_timestamp )
                        all_HittedBin_IDs.append( matchedBin.ID )
                        all_EEX_values.append( current_EEX )
                        all_EEY_values.append( current_EEY )
                        all_Pedersen_values.append( current_PED )
                        all_Hall_values.append( current_HAL )                        
                        Matches += 1
                    else:
                        print( "PARADOX at:", current_MLT, current_MagLat, current_Altitude, current_Kp, " :: ", time_idx, lev_idx, lat_idx, lon_idx )
                        
    # clean up
    print( Matches, "satellite positions where matched inside bins." )
    try:
        CDFroot.close()
    except:
        print (".")
    

    

# Finds the neighbor values of <aValue> inside the list.    
# aList: a list of floats, with ascending sorting
# aValue: a float value
# RETURNS:
#    the value of the lesser Neighbor, the value of the greater Neighbor, the index of the lesser Neighbor, the index of the greater Neighbor, 
def findNeighborValues( aList, aValue ):
    listlength = len(aList)
    stop_idx = -1
    for i in range( 0, listlength ):
        if aValue < aList[i]:
            stop_idx = i
            break
    if stop_idx == -1: # <aValue> is greater than all the values of the list
        LesserNeighborIdx  = listlength - 1
        GreaterNeighborIdx = 0
    elif stop_idx == 0: # <aValue> is lesser than all the values of the list
        LesserNeighborIdx  = listlength - 1
        GreaterNeighborIdx = 0
    else:
        LesserNeighborIdx  = stop_idx-1
        GreaterNeighborIdx = stop_idx
    #
    return LesserNeighborIdx, GreaterNeighborIdx, aList[LesserNeighborIdx], aList[GreaterNeighborIdx]
        
    
    
def CalculateJouleHeatingPerBin():
    for B in Bins:
        if len(B.JH_values) > 0: 
            for aJHvalue in B.JH_values:
                if B.JH_min > aJHvalue: B.JH_min = aJHvalue
                if B.JH_max < aJHvalue: B.JH_max = aJHvalue
                B.JH_mean += aJHvalue
            B.JH_mean = B.JH_mean / len(B.JH_values)
            # for Variance:
            for aJHvalue in B.JH_values:
                B.JH_variance += abs(aJHvalue - B.JH_mean)**2
            B.JH_variance = B.JH_variance / len(B.JH_values)
            #print( "|||||||||||||| ",  statistics.mean(B.JH_values), B.JH_mean )
            #print( ":::::::::::::: ",  statistics.variance(B.JH_values), B.JH_variance,  B.JH_variance/len(B.JH_values))
            

        
        

#################### EVENT LISTENERS ###########################
def Exec_Btn_Clicked( b ):
    global Bins
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilesPath, CALCULATIONS_TIEGCMfolder, CALCULATIONS_ExecutionDuration
    CALCULATIONS_Title         = ExecutionTitle_Text.value
    CALCULATIONS_Description   = ExecutionDescr_Text.value
    CALCULATIONS_RegionName    = BinGroups_Dropdown.value
    CALCULATIONS_OrbitFilesPath = ""
    CALCULATIONS_TIEGCMfolder      = tiegcmFolder_Dropdown.value
    # remove all other bin-groups so that calculation is faster
    newBins = list()
    for B in Bins:
        if B.ID.startswith( BinGroups_Dropdown.value ): newBins.append( B )
    Bins = newBins
    CALCULATIONS_RegionName = Bins[0].Description + " (" + Bins[0].ID[:3] + ")"
    # do it
    startSecs = time.time()
    print( "Calculation for TIEGCM grid started.", datetime.now() )
    ResultsFilename = AssignValuesPerBin_MultipleResultFiles(CALCULATIONS_TIEGCMfolder)  #AssignJouleHeatingValuesToBins( CALCULATIONS_TIEGCMfolder )
    CalculateJouleHeatingPerBin()
    finishSecs = time.time()   
    CALCULATIONS_ExecutionDuration = finishSecs-startSecs
    # print info
    print( "Duration", CALCULATIONS_ExecutionDuration, "seconds." )
    print( "Calculation for TIEGCM grid finshed in " + str(CALCULATIONS_ExecutionDuration) + " seconds." )
    print( "RESULTS (stored in " + ResultsFilename + "):" )
    for B in Bins:
        B.printMe()
    # re-initialize the bins
    InitializeBins()
    print( "Please re-run the notebook and load the results in order to plot them." )

        
def Exec_Btn_alongOrbit_Clicked( b ):
    global Bins
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilesPath, CALCULATIONS_TIEGCMfolder, CALCULATIONS_ExecutionDuration
    CALCULATIONS_Title          = ExecutionTitle_Text.value
    CALCULATIONS_Description    = ExecutionDescr_Text.value
    CALCULATIONS_RegionName     = BinGroups_Dropdown.value
    CALCULATIONS_OrbitFilesPath = OrbitFilesPath_Dropdown.value
    CALCULATIONS_TIEGCMfolder       = tiegcmFolder_Dropdown.value
    ResultsFilename = DaedalusGlobals.CoverageResults_Files_Path + BinGroups_Dropdown.value + "." + CALCULATIONS_TIEGCMfolder[CALCULATIONS_TIEGCMfolder[:-1].rfind('/')+1:-1] + "." + CALCULATIONS_OrbitFilesPath[CALCULATIONS_OrbitFilesPath[:-1].rfind('/')+1:-1] + ".ValuesPerBinResults.nc"
    if path.exists( ResultsFilename ):
        print( "File " + ResultsFilename + " already exists. Cannot continue in order to prevent overwriting useful data." )
    else:
        # remove all other bin-groups so that calculation is faster
        newBins = list()
        for B in Bins:
            if B.ID.startswith( BinGroups_Dropdown.value ): newBins.append( B )
        Bins = newBins
        # calculate
        print( "Joule-Heating-per-Bin-Along-Orbit calculation started.", datetime.now() )
        print( "Reading TIEGCM file from", CALCULATIONS_TIEGCMfolder )
        print( "Results will be stored in", ResultsFilename, "\n" )
        startSecs = time.time()
        AssignJouleHeatingValuesToBins_AlongOrbit( CALCULATIONS_TIEGCMfolder, CALCULATIONS_OrbitFilesPath )
        CalculateJouleHeatingPerBin()
        finishSecs = time.time()   
        CALCULATIONS_ExecutionDuration = finishSecs-startSecs
        #SaveResults_TXT( ResultsFilename ) 
        SaveResults_CDF( ResultsFilename.replace(".txt", ".nc"), "" ) 
        # print info
        print( "Duration", CALCULATIONS_ExecutionDuration, "seconds." )
        print( "Joule-Heating-per-Bin Calculation finshed in " + str(CALCULATIONS_ExecutionDuration) + " seconds." )
        print( "RESULTS (stored in " + ResultsFilename + "):" )
        for B in Bins:
            B.printMe()
        # re-initialize the bins
        InitializeBins()




        
def Load_Btn_Clicked( b ):
    global SELECTED_VARIABLE, SELECTED_VARIABLE_longname, SELECTED_VARIABLE_shortname, SELECTED_VARIABLE_units  
    # set the selected variable as user has chosen
    if Variable_DropDown.value == "Joule Heating":
        SELECTED_VARIABLE = "Ohmic"
        SELECTED_VARIABLE_longname  = "Joule Heating"
        SELECTED_VARIABLE_shortname = "JH"
        SELECTED_VARIABLE_units     = "W/m3"        
    elif Variable_DropDown.value == "Electric Field North":
        SELECTED_VARIABLE = "EEY_si"
        SELECTED_VARIABLE_longname  = "Electric Field North"
        SELECTED_VARIABLE_shortname = "EF(N)"
        SELECTED_VARIABLE_units     = "mV/m"
    elif Variable_DropDown.value == "Electric Field East":
        SELECTED_VARIABLE = "EEX_si"
        SELECTED_VARIABLE_longname  = "Electric Field East"
        SELECTED_VARIABLE_shortname = "ED(E)"
        SELECTED_VARIABLE_units     = "mV/m"
    elif Variable_DropDown.value == "Pedersen Conductivity":
        SELECTED_VARIABLE = "SIGMA_PED"
        SELECTED_VARIABLE_longname  = "Pedersen Conductivity"
        SELECTED_VARIABLE_shortname = "Pedersen"
        SELECTED_VARIABLE_units     = "S/m"
    elif Variable_DropDown.value == "Hall Conductivity":
        SELECTED_VARIABLE = "SIGMA_HAL"
        SELECTED_VARIABLE_longname  = "Hall Conductivity"
        SELECTED_VARIABLE_shortname = "Hall"
        SELECTED_VARIABLE_units     = "S/m"
    elif Variable_DropDown.value == "Convection Heating":
        SELECTED_VARIABLE = "Convection_heating"
        SELECTED_VARIABLE_longname  = "Convection Heating"
        SELECTED_VARIABLE_shortname = "Conv.h."
        SELECTED_VARIABLE_units     = "W/m3"
    elif Variable_DropDown.value == "Wind Heating":
        SELECTED_VARIABLE = "Wind_heating"
        SELECTED_VARIABLE_longname  = "Wind Heating"
        SELECTED_VARIABLE_shortname = "Wind.h."
        SELECTED_VARIABLE_units     = "W/m3"            
    #### Load
    if SavedFilenames_Dropdown.value.endswith( ".txt" ):
        LoadResults( SavedFilenames_Dropdown.value )
    #### Plot
    if Plot_JHdistribution_Checkbox.value == True:
        if SavedFilenames_Dropdown.value.endswith( ".nc" ) or SavedFilenames_Dropdown.value.endswith( "/" ): 
            LoadResults_CDF( SavedFilenames_Dropdown.value, loadTimeValues=False, loadMagLatValues=False, loadMLTvalues=False, loadAltValues=False, loadLatValues=False, loadKpValues=False )
        Plot_Alex_Distribution()
        Plot_JH_Distribution_perBin()
    elif Plot_JHvsMagLat_Checkbox.value==True or Plot_JHvsMLT_Checkbox.value==True or Plot_JHvsAltitude_Checkbox.value==True or Plot_AltitudeVsMagLat_Checkbox.value==True:
        if SavedFilenames_Dropdown.value.endswith( ".nc" )  or SavedFilenames_Dropdown.value.endswith( "/" ): 
            LoadResults_CDF( SavedFilenames_Dropdown.value )
        plotAll()        
        plotAll_perKp()
    elif Plot_AltitudeProfiles_Checkbox.value == True:
        if SavedFilenames_Dropdown.value.endswith( ".nc" ) or SavedFilenames_Dropdown.value.endswith( "/" ): 
            LoadResults_CDF( SavedFilenames_Dropdown.value, loadBinValues=False, loadTimeValues=False, loadMagLatValues=False, loadAltValues=True, loadLatValues=False )
        #plotAltitudeProfiles_perSeason( False )
        #plotAltitudeProfiles_perSeason( True )
        plotAltitudeProfiles_perKpRange()
    elif Plot_HeightIntegrated_Checkbox.value == True:
        plotHeightIntegrated_perKpRange()
        
        
def CompareResults_Btn_Clicked( b ):
    plotComparisonOfResults()
            
################################################################

def tiegcmFolder_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        return

        
def SavedFilenames_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        file_size_in_Gigabytes = os.stat(change['new']).st_size / 1024 / 1024 / 1024
        if file_size_in_Gigabytes > 5:
            Warning_HTML.value= "<b><font color='red'>File size is " + '{0:.1f}'.format(file_size_in_Gigabytes) + " Gigabyte. Ploting will take several minutes.</b>" 
            Warning_HTML.visible=True
        elif file_size_in_Gigabytes > 1:
            Warning_HTML.value= "<b><font color='red'>File size is " + '{0:.1f}'.format(file_size_in_Gigabytes) + " Gigabyte. Ploting will take several seconds.</b>" 
            Warning_HTML.visible=True            
        else:
            Warning_HTML.visible=False



def MainTab_Changed( change ):
    if change['type']=='change' and change['name']=='selected_index':
        if change['new'] == 0:
            change = {'type':'change', 'name':'value', 'new':tiegcmFolder_Dropdown.value}
        else:
            change = {'type':'change', 'name':'value', 'new':SavedFilenames_Dropdown.value}
        tiegcmFolder_Dropdown_onChange( change )  
            
def createGUI():
    ## the top level visual elements
    MainPanel = w.VBox()    
    MainTab = w.Tab() 
    LoadCoveragePanel           = w.VBox()
    CalcCoveragePanel           = w.VBox()
    CalcCoverageAlongOrbitPanel = w.VBox()
    CompareResultsPanel         = w.VBox()
    ## the checkboxes which allow user to select which plots he wants to create
    PlotSelectionPanel = w.VBox()
    PlotSelectionPanel.children = [Plot_JHvsMagLat_Checkbox, Plot_JHvsMLT_Checkbox, Plot_JHvsAltitude_Checkbox, Plot_AltitudeVsMagLat_Checkbox, w.HBox([Plot_JHdistribution_Checkbox, RegressionOptions_Dropdown]), Plot_AltitudeProfiles_Checkbox, Plot_HeightIntegrated_Checkbox ]
    ##
    MainTab.children = [ CalcCoverageAlongOrbitPanel, CalcCoveragePanel, LoadCoveragePanel ]
    MainTab.set_title(0, 'Calc along Orbit')
    MainTab.set_title(1, 'Calc for TIEGCMgrid')
    MainTab.set_title(2, 'Load Results')
    MainTab.set_title(3, 'Compare Results')
    MainPanel.children = [ MainTab, OrbitPreviewImage ]    
    ## 
    Exec_Btn = w.Button (description='Calculate for TIEGCM grid',tooltip="Click here to calculate",)
    Exec_Btn.style.button_color = 'MediumTurquoise'
    Exec_Btn.on_click( Exec_Btn_Clicked )
    CalcCoveragePanel.children = [tiegcmFolder_Dropdown, BinGroups_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, Exec_Btn ] # I removed PlotSelectionPanel
    ##
    ExecAlongOrbit_Btn = w.Button (description='Calc along Orbit',tooltip="Click here to calculate",)
    ExecAlongOrbit_Btn.style.button_color = 'Coral'
    ExecAlongOrbit_Btn.on_click( Exec_Btn_alongOrbit_Clicked )
    CalcCoverageAlongOrbitPanel.children = [tiegcmFolder_Dropdown, BinGroups_Dropdown, OrbitFilesPath_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, ExecAlongOrbit_Btn ] # I removed PlotSelectionPanel
    ##
    Load_Btn = w.Button (description='Load Results from',tooltip="Click here to plot",)
    Load_Btn.style.button_color = 'YellowGreen'
    Load_Btn.on_click( Load_Btn_Clicked )
    L2_horizontal = w.HBox()
    L2_horizontal.children = [Load_Btn, SavedFilenames_Dropdown]
    LoadCoveragePanel.children = [ w.HBox([Load_Btn, SavedFilenames_Dropdown]), Variable_DropDown, Warning_HTML, PlotSelectionPanel ]
    ##
    CompareResults_Btn = w.Button (description='Compare Results',tooltip="Click here to plot",)
    CompareResults_Btn.style.button_color = 'Gold'
    CompareResults_Btn.on_click( CompareResults_Btn_Clicked )
    CompareResultsPanel.children = [ w.HBox([w.HTML(value="TIEGCM results:"),SavedFilenames_Dropdown]), w.HBox([w.HTML(value="Orbit results:"),SavedFilenames2_Dropdown]), CompareResults_Btn ]
    ## Assign event listeners
    tiegcmFolder_Dropdown.observe( tiegcmFolder_Dropdown_onChange )
    SavedFilenames_Dropdown.observe( SavedFilenames_Dropdown_onChange )
    MainTab.observe( MainTab_Changed )
    ## display orbit-related image
    change = {'type':'change', 'name':'value', 'new':tiegcmFolder_Dropdown.value}
    tiegcmFolder_Dropdown_onChange( change )        
    return MainPanel
# PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT 
# PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT 
# PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT 

def plotAll():
    # choose which bins we are going to work with
    if "(" in CALCULATIONS_RegionName:
        RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    else:
        RegionID = CALCULATIONS_RegionName
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )

    # remember the Kp ranges of these bins. Each Kp-range will have its own sub-plot
    TMP_KpRanges = list()
    for B in BinsIncludedAtPlot:
        if [B.Kp_min, B.Kp_max] not in TMP_KpRanges: TMP_KpRanges.append( [B.Kp_min, B.Kp_max] )  
            
    # --- init various plotting parameters ---
    max_num_of_points = 10000
    plot_step = int(  len(all_JH_values) / max_num_of_points  )
    if plot_step <= 0: plot_step = 1
    n = max_num_of_points
    if n > len(all_JH_values):  n = len(all_JH_values)
    print( "I will plot", n, "out of", len(all_JH_values), "points (1 per", plot_step, ")")
    TMP_JH_values       = list()
    TMP_MagLat_values   = list()
    TMP_MLT_values      = list()
    TMP_Altitude_values = list()
    for idx in range( 0, len(all_JH_values) ):
        if int(idx / 1) % int(plot_step) == 0: 
            TMP_JH_values.append( all_JH_values[idx] )
            TMP_MagLat_values.append( all_MagLat_values[idx] )
            TMP_MLT_values.append( all_MLT_values[idx] )
            TMP_Altitude_values.append( all_Altitude_values[idx] )
    
    # handle MLT ranges like 22:00-02:00
    MLT_min_toPlot = BinsIncludedAtPlot[0].MLT_min
    MLT_max_toPlot = BinsIncludedAtPlot[0].MLT_max
    if BinsIncludedAtPlot[0].MLT_min > BinsIncludedAtPlot[0].MLT_max:
        MLT_max_toPlot += 24
        for i in range(0, len(TMP_MLT_values)):
            if TMP_MLT_values[i] < BinsIncludedAtPlot[0].MLT_min: TMP_MLT_values[i] += 24
    # define altitude range for X-axis
    Altitude_max_toPlot = max(all_Altitude_values)
    if Altitude_max_toPlot < 140: Altitude_max_toPlot = 140
                
    # define max JH value to be plotted
    if SELECTED_VARIABLE == "Ohmic" or SELECTED_VARIABLE == "Convection_heating":
        JHmax = 1.4e-7
        if CALCULATIONS_RegionName.startswith( "SQ" ): JHmax = max(all_JH_values)
    else:
        JHmax = max(all_JH_values)
        
    if len(all_MagLat_values) > 0  and  Plot_JHvsMagLat_Checkbox.value==True:
        print( "Plotting ", len(TMP_MagLat_values), "points" )
        MyColorsIndex = 0
        fig = go.Figure()        
        fig.add_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=TMP_MagLat_values, y=TMP_JH_values, mode='markers', marker_size=2) )
        BinAnnotations = list()
        prevKpMin = -1
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for mean line
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        
                # add visuals for the mean line
                fig.add_shape( type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=2, ), )    
                # add info as legend for this bin
                fig.add_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)) , x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*3/4, y=B.JH_mean, xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                # add visuals for standard deviation
                fig.add_shape( type="line", x0=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=1, ), ) 
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" vs Magnetic Latitude - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        pass
        #print( "There are no points for MagLat plot" )                

    if len(all_MLT_values) > 0  and  Plot_JHvsMLT_Checkbox.value == True:
        MyColorsIndex = 0
        fig = go.Figure()
        print( "Plotting ", len(TMP_MLT_values), "points" )
        fig.add_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=TMP_MLT_values, y=TMP_JH_values, mode='markers', marker_size=2) )
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for mean line
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        
                # add visuals for the mean line             
                fig.add_shape( type="line", x0=MLT_min_toPlot, y0=B.JH_mean,     x1=MLT_max_toPlot, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=2, ), )    
                # add info as legend for this bin
                fig.add_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*3/4, y=B.JH_mean, xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                # add visuals for standard deviation
                fig.add_shape( type="line", x0=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" vs Magnetic Local Time - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[MLT_min_toPlot, MLT_max_toPlot], title="Magnetic Local Time (hours)") #fig.update_xaxes(range=[min(TMP_MLT_values), max(TMP_MLT_values)], title="Magnetic Local Time (hours)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        pass
        #print( "There are no points for MLT plot" )        
    
    if len(TMP_Altitude_values) > 0  and  Plot_JHvsAltitude_Checkbox.value == True:
        MyColorsIndex = 0
        fig = go.Figure()
        print( "Plotting ", len(TMP_Altitude_values), "points" )
        fig.add_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=TMP_Altitude_values, y=TMP_JH_values, mode='markers', marker_size=2) )
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for mean line
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        
                # add visuals for the mean line
                fig.add_shape( type="line", x0=B.Altitude_min, y0=B.JH_mean,     x1=B.Altitude_max, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=2, ), )    
                # add info as legend for this bin
                fig.add_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*3/4, y=B.JH_mean, xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                # add visuals for standard deviation
                fig.add_shape( type="line", x0=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" vs Altitude - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[115, Altitude_max_toPlot], title="Altitude (km)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        pass
        #print( "There are no points for Altitude plot" )
    
    
    if len(TMP_JH_values) > 0  and  Plot_AltitudeVsMagLat_Checkbox.value == True:
        MyColorsIndex = 0
        fig = go.Figure()
        print( "Plotting ", len(TMP_JH_values), "points" )
        
        colorMean = 0
        for n in TMP_JH_values: colorMean += n
        colorMean = float( colorMean / len(TMP_JH_values) )
        colorMin = float(colorMean / 10)
        colorMax = float(colorMean * 10)
        fig.add_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=TMP_MagLat_values, y=TMP_Altitude_values, mode='markers', 
                       marker=dict( size=2, color=TMP_JH_values, colorscale="Jet", cmin=colorMin, cmax=colorMax, colorbar=dict(title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")" )) ) )
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                #fig.add_shape( type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=1, ), )    
                #fig.add_trace( go.Scatter(name="Bin Mean: " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + " <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b> Variance=" + str(B.JH_variance), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                MyColorsIndex += 1
                if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
        fig.update_layout( title="Altitude vs Magnetic Latitude - " + CALCULATIONS_RegionName, 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees)" )
        fig.update_yaxes(range=[min(all_Altitude_values), max(all_Altitude_values)], title="Altitude(km)")
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        pass
        #print( "There are no points for Altitude-MagLat plot" )                

    

# PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP 
# PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP 
# PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP 

def plotAll_perKp():
    # choose which bins we are going to work with
    if "(" in CALCULATIONS_RegionName:
        RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    else:
        RegionID = CALCULATIONS_RegionName
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )          

    # --- init various plotting parameters ---
    # handle MLT ranges like 22:00-02:00
    MLT_min_toPlot = BinsIncludedAtPlot[0].MLT_min
    MLT_max_toPlot = BinsIncludedAtPlot[0].MLT_max
    if BinsIncludedAtPlot[0].MLT_min > BinsIncludedAtPlot[0].MLT_max:
        MLT_max_toPlot += 24
        for i in range(0, len(all_MLT_values)):
            if all_MLT_values[i] < BinsIncludedAtPlot[0].MLT_min: all_MLT_values[i] += 24
        for B in BinsIncludedAtPlot:
            for i in range(0, len(B.MLT_values)):
                if B.MLT_values[i] < BinsIncludedAtPlot[0].MLT_min: B.MLT_values[i] += 24
    # define altitude range for X-axis
    Altitude_max_toPlot = max(all_Altitude_values)
    if Altitude_max_toPlot < 140: Altitude_max_toPlot = 140
    # define max JH value to be plotted
    if SELECTED_VARIABLE == "Ohmic" or SELECTED_VARIABLE == "Convection_heating":
        JHmax = 1.4e-7
        if CALCULATIONS_RegionName.startswith( "SQ" ): JHmax = max(all_JH_values)
    else:
        JHmax = max(all_JH_values)

    # CONSTRUCT DATA per Kp-range
    # remember the Kp ranges of the plot's bins. Each Kp-range will have its own sub-plot
    All_KpRanges = list()
    for B in BinsIncludedAtPlot:
        if [B.Kp_min, B.Kp_max] not in All_KpRanges: 
            All_KpRanges.append( [B.Kp_min, B.Kp_max] )  
    # group data according to Kp-range
    JH_values_perKp       = list() # 2d-list: one row for each Kp-range
    MagLat_values_perKp   = list() # 2d-list: one row for each Kp-range
    MLT_values_perKp      = list() # 2d-list: one row for each Kp-range
    Altitude_values_perKp = list() # 2d-list: one row for each Kp-range
    Time_values_perKp     = list()
    for i in range(0, len(All_KpRanges)): 
        JH_values_perKp.append( list() )
        MagLat_values_perKp.append( list() )
        MLT_values_perKp.append( list() )
        Altitude_values_perKp.append( list() )
        Time_values_perKp.append( list() )
        for B in BinsIncludedAtPlot:
            if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: 
                JH_values_perKp[i]       += B.JH_values
                MagLat_values_perKp[i]   += B.MagLat_values
                MLT_values_perKp[i]      += B.MLT_values
                Altitude_values_perKp[i] += B.Altitude_values
                Time_values_perKp[i]     += B.Time_values
    # make the data set smaller so that it can be plotted
    max_num_of_points = 80000
    print( "\n" ) 
    for i in range(0, len(All_KpRanges)):
        plot_step = int(  len(JH_values_perKp[i]) / max_num_of_points  )
        n = max_num_of_points 
        if n > len(JH_values_perKp[i]):  n = len(JH_values_perKp[i])
        print( "I will plot", n, "out of", len(JH_values_perKp[i]), "points (1 per", plot_step, ")" + " for " + str(All_KpRanges[i][0]) + "<Kp<" + str(All_KpRanges[i][1]) )
        if plot_step > 0:
            #JH_values_perKp[i]       = JH_values_perKp[i][0::plot_step]
            #MagLat_values_perKp[i]   = MagLat_values_perKp[i][0::plot_step]
            #MLT_values_perKp[i]      = MLT_values_perKp[i][0::plot_step]
            #Altitude_values_perKp[i] = Altitude_values_perKp[i][0::plot_step]
            TMP_JH_values       = list()
            TMP_MagLat_values   = list()
            TMP_MLT_values      = list()
            TMP_Altitude_values = list()
            TMP_Time_values     = list()
            for idx in range( 0, len(JH_values_perKp[i]) ):
                if int(idx / 1) % int(plot_step) == 0: 
                    TMP_JH_values.append( JH_values_perKp[i][idx] )
                    TMP_MagLat_values.append( MagLat_values_perKp[i][idx] )
                    TMP_MLT_values.append( MLT_values_perKp[i][idx] )
                    TMP_Altitude_values.append( Altitude_values_perKp[i][idx] )
                    try:
                        TMP_Time_values.append( Time_values_perKp[i][idx] )
                    except:
                        pass
            JH_values_perKp[i]       = TMP_JH_values
            MagLat_values_perKp[i]   = TMP_MagLat_values
            MLT_values_perKp[i]      = TMP_MLT_values
            Altitude_values_perKp[i] = TMP_Altitude_values
            Time_values_perKp[i] = TMP_Time_values
            
    # PLOT
    if len(all_MagLat_values) > 0  and  Plot_JHvsMagLat_Checkbox.value==True:
        fig = make_subplots(rows=len(All_KpRanges), cols=1, shared_xaxes=False, vertical_spacing=0.05)
        for i in range(0, len(All_KpRanges)):
            fig.append_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=MagLat_values_perKp[i], y=JH_values_perKp[i], mode='markers', marker_size=2, marker_color=MyColors[i]), row=i+1, col=1 )
        #
        BinAnnotations = list()
        FigureShapes = list()
        MyColorsIndex = 0
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose which sub-plot will host this Bin's data
                SubPlotIdx = 1
                for i in range(0, len(All_KpRanges)):
                    if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: SubPlotIdx = i+1
                # choose color for mean line
                MyColorsIndex = SubPlotIdx - 1
                # add visuals for the mean line
                FigureShapes.append( dict(type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, ),  xref= 'x'+str(SubPlotIdx), yref= 'y'+str(SubPlotIdx))  )  #fig.append_shape( dict(type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, )), row=SubPlotIdx, col=1 )    
                # add info as legend for this bin
                fig.append_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]), row=SubPlotIdx, col=1 )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*3/4, y=B.JH_mean, text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex]), xref='x1', yref='y'+str(SubPlotIdx) ) )
                # add visuals for standard deviation
                FigureShapes.append( dict(type="line", x0=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=2, ), xref= 'x1', yref='y'+str(SubPlotIdx) )  )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout(shapes=FigureShapes)
        fig.update_layout( title=SELECTED_VARIABLE_longname+" vs Magnetic Latitude - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=1500, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        for i in range(0, len(All_KpRanges)):
            fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=i+1, col=1 )
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for MagLat per-Kp-range plot" )                
    
    ##
    if len(all_MLT_values) > 0  and  Plot_JHvsMLT_Checkbox.value == True:
        fig = make_subplots(rows=len(All_KpRanges), cols=1, shared_xaxes=False, vertical_spacing=0.05)
        for i in range(0, len(All_KpRanges)):
            fig.append_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=MLT_values_perKp[i], y=JH_values_perKp[i], mode='markers', marker_size=2, marker_color=MyColors[i]), row=i+1, col=1 )
        #
        BinAnnotations = list()
        FigureShapes = list()
        MyColorsIndex = 0
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose which sub-plot will host this Bin's data
                SubPlotIdx = 1
                for i in range(0, len(All_KpRanges)):
                    if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: SubPlotIdx = i+1
                    #fig.update_xaxes(range=[min(MLT_values_perKp), max(MLT_values_perKp)], title="Magnetic Local Time (hours) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=SubPlotIdx, col=1 )                
                # choose color for mean line
                MyColorsIndex = SubPlotIdx - 1
                # add visuals for the mean line             
                FigureShapes.append( dict(type="line", x0=MLT_min_toPlot, y0=B.JH_mean,     x1=MLT_max_toPlot, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, ),  xref= 'x'+str(SubPlotIdx), yref= 'y'+str(SubPlotIdx))  ) 
                # add info as legend for this bin
                fig.append_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]), row=SubPlotIdx, col=1 )
                # add bin name above the mean line
                BinAnnotations.append(          dict( x=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*3/4, y=B.JH_mean, text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex]), xref='x1', yref='y'+str(SubPlotIdx)) )
                FigureShapes.append( dict(type="line", x0=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=2, ), xref= 'x1', yref= 'y'+str(SubPlotIdx) )  )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( shapes=FigureShapes )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" vs Magnetic Local Time - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=1500, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        #fig.update_xaxes(range=[MLT_min_toPlot, MLT_max_toPlot], title="Magnetic Local Time (hours)")
        for i in range(0, len(All_KpRanges)):
            fig.update_xaxes(range=[MLT_min_toPlot, MLT_max_toPlot], title="Magnetic Local Time (hours) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=i+1, col=1 )
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for MLT per-Kp-range plot" )        
    
    if len(all_Altitude_values) > 0  and  Plot_JHvsAltitude_Checkbox.value == True:
        # the main scatter plot
        fig = make_subplots(rows=len(All_KpRanges), cols=1, shared_xaxes=False, vertical_spacing=0.05)
        for i in range(0, len(All_KpRanges)):
            fig.append_trace( go.Scatter(name=SELECTED_VARIABLE_longname, x=Altitude_values_perKp[i], y=JH_values_perKp[i], mode='markers', marker_size=2, marker_color=MyColors[i]), row=i+1, col=1 )
        # lines along neighbor points (according to time, only for orbit results)
        if len(CALCULATIONS_OrbitFilesPath) > 0: 
            neighbors_JH   = list()
            neighbors_Alt = list()
            for i in range(0, len(All_KpRanges)):
                for t in range(0, len(Time_values_perKp[i])):
                    if t>0 and Time_values_perKp[i][t]-Time_values_perKp[i][t-1]<=10: # orbit file has 1 entry per 10 sec
                        if len(neighbors_JH)==0:
                            neighbors_JH.append( JH_values_perKp[i][t-1] )
                            neighbors_Alt.append( Altitude_values_perKp[i][t-1] )
                        neighbors_JH.append( JH_values_perKp[i][t] )
                        neighbors_Alt.append( Altitude_values_perKp[i][t] )
                    else:
                        fig.append_trace( go.Scatter(x=neighbors_Alt, y=neighbors_JH, mode='lines', line_width=1, line_color="rgba("+Hex_to_RGB(MyColors[i])+", 0.18)", showlegend=False ), row=i+1, col=1)
                        neighbors_JH   = list()
                        neighbors_Alt = list()
        # annotations, shapes etc
        BinAnnotations = list()
        FigureShapes = list()
        MyColorsIndex = 0
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose which sub-plot will host this Bin's data
                SubPlotIdx = 0
                for i in range(0, len(All_KpRanges)):
                    if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: SubPlotIdx = i+1
                    fig.update_xaxes(range=[115, Altitude_max_toPlot], title="Altitude (km) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=i+1, col=1 )
                # choose color for mean line
                MyColorsIndex = SubPlotIdx - 1                
                # add visuals for the mean line
                FigureShapes.append( dict(type="line", x0=B.Altitude_min, y0=B.JH_mean,     x1=B.Altitude_max, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, ),  xref= 'x'+str(SubPlotIdx), yref= 'y'+str(SubPlotIdx))  )                
                # add info as legend for this bin
                fig.append_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]), row=SubPlotIdx, col=1 )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*3/4, y=B.JH_mean, text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex]), xref='x1', yref='y'+str(SubPlotIdx) ) )
                # add visuals for standard deviation
                FigureShapes.append( dict(type="line", x0=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=2, ), xref= 'x1', yref= 'y'+str(SubPlotIdx) )  )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( shapes=FigureShapes )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" vs Altitude - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=1500, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        #fig.update_xaxes(range=[115, Altitude_max_toPlot], title="Altitude (km)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for Altitude per-Kp-range plot" )
    
    

    
    
    
    
    
# Y = a*X^k + c
def func_powerlaw(x,  a, k, c):
    return a * (x**k)  +  c

# Y = a * log(X) + c
def func_logarithmic(x,  a, c):
    return [ (a * math.log(x_i) + c)  for x_i in x ]

# Y = a / e ^ (bx) + c
def func_euler(x,  a, b, c):
    return [ (a / (math.e**(b*x_i)) + c) for x_i in x ]

def func_maxwellian(x,  a, b, c):
    return [ (a * x_i*x_i * (math.e**(-b*x_i)) + c) for x_i in x ]

def Plot_JH_Distribution_perBin():
    num_of_slots = 20
    # choose which bins we are going to work with
    if "(" in CALCULATIONS_RegionName:
        RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    else:
        RegionID = CALCULATIONS_RegionName
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )
    
    # init 
    #generalMean = sum(all_JH_values) / len(all_JH_values)
    upper_value = (2)*BinsIncludedAtPlot[0].JH_mean
    lower_value = 0
    #upper_value = max(all_JH_values)
    #lower_value = min(all_JH_values)
    if lower_value > upper_value: # negative mean
        tmp = lower_value
        lower_value = upper_value
        upper_value = tmp
    slot_length = (upper_value - lower_value) / num_of_slots
    if slot_length == 0: 
        print( "No values for Distribution Plot" )
        return
    
    # calculate distribution for each bin
    for B in BinsIncludedAtPlot:
        B.JH_distribution = [0] * num_of_slots
        #print(B.ID, "distribution:")
        for aJHval in B.JH_values:
            if aJHval >= lower_value  and    aJHval <= upper_value:
                slot_idx = int(   (aJHval - lower_value) / slot_length  )
            else:
                continue
            #print( ">>>>>> ", slot_idx, len(B.JH_distribution) )
            if  slot_idx >= len(B.JH_distribution): slot_idx = num_of_slots-1
            B.JH_distribution[ slot_idx ] += 1
        #print(B.JH_distribution, "\n")    
    
    # Normalize the distribution to [0,1] at y-axis
    #for B in BinsIncludedAtPlot:
    #    num_of_all_points_in_bin_distribution = sum(B.JH_distribution)
    #    for slot_idx in range(0, len(B.JH_distribution)):
    #        B.JH_distribution[ slot_idx ]  /=  num_of_all_points_in_bin_distribution
    
    # plot the distribution of all bins on the same figure
    if len(all_JH_values) > 0  and  Plot_JHdistribution_Checkbox.value == True:
        MyColorsIndex = 0
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        fig = go.Figure()        
        print( "Plotting " + SELECTED_VARIABLE_longname + " Distribution" )
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for this bin's points
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        

                if RegressionOptions_Dropdown.value.startswith( "Polynomial" ):
                    # calculate the Polynomial Regression
                    degree = int( RegressionOptions_Dropdown.value[-1] )
                    myPolynomial = np.polyfit( list(range(0,num_of_slots)), B.JH_distribution, degree )
                    # construct the equation to display
                    poly_str = "y = "
                    for i in range(0, len(myPolynomial)): 
                        if i>0 and myPolynomial[i] > 0: poly_str += "+ "
                        poly_str += "{:.2e}".format(myPolynomial[i])
                        if i < len(myPolynomial)-1: poly_str += "x^" + str(len(myPolynomial)-1-i) + " "
                    # draw the Polynomial Regression
                    mymodel = np.poly1d(myPolynomial)
                    myline = np.linspace(1, num_of_slots, num_of_slots)
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=myline, y=mymodel(myline), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                elif RegressionOptions_Dropdown.value == "Power law":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_powerlaw, list(range(0,num_of_slots)), B.JH_distribution)
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * x^" + "{:.2e}".format(OptimalParams[1]) + " + " + "{:.2e}".format(OptimalParams[2])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(0,num_of_slots)), y=func_powerlaw(list(range(0,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                                               
                elif RegressionOptions_Dropdown.value == "Logarithmic":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_logarithmic, list(range(1,num_of_slots)), B.JH_distribution[1:])
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * log(x) +" + "{:.2e}".format(OptimalParams[1])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(1,num_of_slots)), y=func_logarithmic(list(range(1,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                        
                elif RegressionOptions_Dropdown.value == "Euler":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_euler, list(range(1,num_of_slots)), B.JH_distribution[1:])
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " / e^(" + "{:.2e}".format(OptimalParams[1]) + "*x) + " + "{:.2e}".format(OptimalParams[2])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(1,num_of_slots)), y=func_euler(list(range(1,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                        
                elif RegressionOptions_Dropdown.value == "Maxwell":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_maxwellian, list(range(1,num_of_slots)), B.JH_distribution[1:])
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * x^2 * e^(-" + "{:.2e}".format(OptimalParams[1]) + "*x) + " + "{:.2e}".format(OptimalParams[2])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(1,num_of_slots)), y=func_maxwellian(list(range(1,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )                    
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                        

                # draw the distribution
                bin_desciption = B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2))
                fig.add_trace( go.Scatter(name=bin_desciption, x=list(range(0,num_of_slots)), y=B.JH_distribution, mode='markers', marker_size=3, marker_color=MyColors[MyColorsIndex]  ) )
                
                # add visuals for the mean line                
                mean_slot_idx = int(   (B.JH_mean - lower_value) / slot_length  )
                fig.add_shape( type="line", x0=mean_slot_idx, y0=0,     x1=mean_slot_idx, y1=(95/100)*max(B.JH_distribution),     line=dict( color=MyColors[MyColorsIndex], width=1, ), )    
                # add bin name above the mean line
                BinAnnotations.append( dict( x=mean_slot_idx, y=(95/100)*max(B.JH_distribution), xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                
                # add visuals for standard deviation
                #StDev_slots_width = int(   ((B.JH_variance)**(1/2)/2) / slot_length  )
                #fig.add_shape( type="line", x0=mean_slot_idx-StDev_slots_width, y0=(95/100)*max(B.JH_distribution),     x1=mean_slot_idx+StDev_slots_width, y1=(95/100)*max(B.JH_distribution),     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                
                BinIdx += 1
                
        # draw correct ticks at the x-axis, containing the JH values
        XaxisTickPositions = list()
        XaxisTickLabels = list()
        for i in range( 0, num_of_slots, int(num_of_slots/5) ):
            XaxisTickPositions.append( i )
            XaxisTickLabels.append(  "{:.3e}".format(lower_value + i*slot_length)  )            
        XaxisTickPositions.append( num_of_slots-1 )
        XaxisTickLabels.append(  "{:.3e}".format(upper_value)  )
        fig.update_xaxes( tickmode = 'array', tickvals=XaxisTickPositions,  ticktext=XaxisTickLabels )
                
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" Distribution per Bin - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=900, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[0,num_of_slots-1], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        fig.update_yaxes(title="Number of hits inside the bin") #rangemode='nonnegative'
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)


        

        
        
# slack msg: May 26
def Plot_Alex_Distribution():
    num_of_slots = 25
    # choose which bins we are going to work with
    if "(" in CALCULATIONS_RegionName:
        RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    else:
        RegionID = CALCULATIONS_RegionName
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )


    print( "%%%%%%%%")
    print( len(BinsIncludedAtPlot) )
    print( BinsIncludedAtPlot[0].JH_mean )
    print( len(BinsIncludedAtPlot[0].JH_values) )
            
            
    # merge all into one bin # TODO: remove after orbit calculation with correct sub-bins
    if ( len(BinsIncludedAtPlot) == 3 ):
        print( "MERGING" )
        #BinsIncludedAtPlot[0].JH_values += BinsIncludedAtPlot[1].JH_values
        #BinsIncludedAtPlot[0].JH_values += BinsIncludedAtPlot[2].JH_values
        concatLists( BinsIncludedAtPlot[0].JH_values, BinsIncludedAtPlot[1].JH_values )
        concatLists( BinsIncludedAtPlot[0].JH_values, BinsIncludedAtPlot[2].JH_values )
        BinsIncludedAtPlot[0].JH_mean = sum(BinsIncludedAtPlot[0].JH_values) / len(BinsIncludedAtPlot[0].JH_values)
        del BinsIncludedAtPlot[-1]
        del BinsIncludedAtPlot[-1]
            
    
    print( "%%%%%%%%")
    print( len(BinsIncludedAtPlot) )
    print( BinsIncludedAtPlot[0].JH_mean )
    print( len(BinsIncludedAtPlot[0].JH_values) )
    print( "%%%%%%%%")
    
    # init 
    #generalMean = sum(all_JH_values) / len(all_JH_values)
    upper_value = (2)*BinsIncludedAtPlot[0].JH_mean
    lower_value = 0
    #upper_value = max(all_JH_values)
    #lower_value = min(all_JH_values)
    if lower_value > upper_value: # negative mean
        tmp = lower_value
        lower_value = upper_value
        upper_value = tmp
        
    upper_value = 1.3e-8
    
    slot_length = (upper_value - lower_value) / num_of_slots
    if slot_length == 0: 
        print( "No values for Distribution Plot" )
        return
    
    # calculate distribution for each bin
    for B in BinsIncludedAtPlot:
        B.JH_distribution = [0] * num_of_slots
        #print(B.ID, "distribution:")
        for aJHval in B.JH_values:
            if aJHval >= lower_value  and    aJHval <= upper_value:
                slot_idx = int(   (aJHval - lower_value) / slot_length  )
            else:
                continue
            #print( ">>>>>> ", slot_idx, len(B.JH_distribution) )
            if  slot_idx >= len(B.JH_distribution): slot_idx = num_of_slots-1
            B.JH_distribution[ slot_idx ] += 1
        #print(B.JH_distribution, "\n")    
    
    # Normalize the distribution to [0,1] at y-axis
    #for B in BinsIncludedAtPlot:
    #    num_of_all_points_in_bin_distribution = sum(B.JH_distribution)
    #    for slot_idx in range(0, len(B.JH_distribution)):
    #        B.JH_distribution[ slot_idx ]  /=  num_of_all_points_in_bin_distribution
    
    # plot the distribution of all bins on the same figure
    if len(all_JH_values) > 0  and  Plot_JHdistribution_Checkbox.value == True:
        MyColorsIndex = 0
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        fig = go.Figure()        
        print( "Plotting " + SELECTED_VARIABLE_longname + " Distribution" )
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for this bin's points
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        

                if RegressionOptions_Dropdown.value.startswith( "Polynomial" ):
                    # calculate the Polynomial Regression
                    degree = int( RegressionOptions_Dropdown.value[-1] )
                    myPolynomial = np.polyfit( list(range(0,num_of_slots)), B.JH_distribution, degree )
                    # construct the equation to display
                    poly_str = "y = "
                    for i in range(0, len(myPolynomial)): 
                        if i>0 and myPolynomial[i] > 0: poly_str += "+ "
                        poly_str += "{:.2e}".format(myPolynomial[i])
                        if i < len(myPolynomial)-1: poly_str += "x^" + str(len(myPolynomial)-1-i) + " "
                    # draw the Polynomial Regression
                    mymodel = np.poly1d(myPolynomial)
                    myline = np.linspace(1, num_of_slots, num_of_slots)
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=myline, y=mymodel(myline), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                elif RegressionOptions_Dropdown.value == "Power law":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_powerlaw, list(range(0,num_of_slots)), B.JH_distribution)
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * x^" + "{:.2e}".format(OptimalParams[1]) + " + " + "{:.2e}".format(OptimalParams[2])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(0,num_of_slots)), y=func_powerlaw(list(range(0,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                                               
                elif RegressionOptions_Dropdown.value == "Logarithmic":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_logarithmic, list(range(1,num_of_slots)), B.JH_distribution[1:])
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * log(x) +" + "{:.2e}".format(OptimalParams[1])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(1,num_of_slots)), y=func_logarithmic(list(range(1,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                        
                elif RegressionOptions_Dropdown.value == "Euler":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_euler, list(range(1,num_of_slots)), B.JH_distribution[1:])
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " / e^(" + "{:.2e}".format(OptimalParams[1]) + "*x) + " + "{:.2e}".format(OptimalParams[2])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(1,num_of_slots)), y=func_euler(list(range(1,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=3) ) )
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                        
                elif RegressionOptions_Dropdown.value == "Maxwell":
                    try:
                        OptimalParams, OptParamsCovariance = curve_fit(func_maxwellian, list(range(1,num_of_slots)), B.JH_distribution[1:])
                        poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * x^2 * e^(-" + "{:.2e}".format(OptimalParams[1]) + "*x) + " + "{:.2e}".format(OptimalParams[2])
                        fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, mode='lines', x=list(range(1,num_of_slots)), y=func_maxwellian(list(range(1,num_of_slots)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )                    
                    except:
                        print( "Warning: Curve fit failed for", B.ID )                        

                # draw the distribution
                bin_desciption = B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2))
                #fig.add_trace( go.Scatter(name=bin_desciption, x=list(range(0,num_of_slots)), y=B.JH_distribution, mode='markers', marker_size=3, marker_color=MyColors[MyColorsIndex]  ) )
                #fig.add_trace( go.Bar(name=bin_desciption, x=list(range(0,num_of_slots)), y=B.JH_distribution, marker_color="LightSkyBlue" ) )
                
                # add visuals for the mean line                
                mean_slot_idx = int(   (B.JH_mean - lower_value) / slot_length  )
                fig.add_shape( type="line", x0=mean_slot_idx, y0=0,     x1=mean_slot_idx, y1=(95/100)*max(B.JH_distribution),     line=dict( color=MyColors[MyColorsIndex], width=1, ), )    
                # add bin name above the mean line
                BinAnnotations.append( dict( x=mean_slot_idx, y=(95/100)*max(B.JH_distribution), xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                
                # add visuals for standard deviation
                #StDev_slots_width = int(   ((B.JH_variance)**(1/2)/2) / slot_length  )
                #fig.add_shape( type="line", x0=mean_slot_idx-StDev_slots_width, y0=(95/100)*max(B.JH_distribution),     x1=mean_slot_idx+StDev_slots_width, y1=(95/100)*max(B.JH_distribution),     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                
                BinIdx += 1
                
        # draw correct ticks at the x-axis, containing the JH values
        XaxisTickPositions = list()
        XaxisTickLabels = list()
        for i in range( 0, num_of_slots, int(num_of_slots/5) ):
            XaxisTickPositions.append( i )
            XaxisTickLabels.append(  "{:.3e}".format(lower_value + i*slot_length)  )            
        XaxisTickPositions.append( num_of_slots-1 )
        XaxisTickLabels.append(  "{:.3e}".format(upper_value)  )
        fig.update_xaxes( tickmode = 'array', tickvals=XaxisTickPositions,  ticktext=XaxisTickLabels )
                
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title=SELECTED_VARIABLE_longname+" Distribution per Bin - " + getBinDescription(CALCULATIONS_RegionName), 
                           width=1000, height=900, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[0,num_of_slots-1], title=SELECTED_VARIABLE_shortname+" ("+SELECTED_VARIABLE_units+")", showexponent = 'all', exponentformat = 'e')
        fig.update_yaxes(title="Number of hits inside the bin") #rangemode='nonnegative'
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
        

        
        
def plotComparisonOfResults():
    ColorTriplet = ["#EEEEEE", "#FF4447", "#257985"] # white red petrol
    # init
    BinIDs = list()
    Means1 = list()
    Means2 = list()
    StDev1 = list()
    StDev2 = list()
    RegionDescription = ""
    # load results no1
    if SavedFilenames_Dropdown.value.endswith( ".nc" ):
        LoadResults_CDF( SavedFilenames_Dropdown.value )
    else:
        LoadResults( SavedFilenames_Dropdown.value )
    for B in Bins:
        if B.JH_mean != 0:
            RegionDescription = B.Description
            BinIDs.append( B.ID )
            Means1.append( B.JH_mean )
            StDev1.append( math.sqrt(B.JH_variance) )
            print( B.ID, B.JH_mean, math.sqrt(B.JH_variance), B.JH_variance**(1/2) )
    # load results no2
    if SavedFilenames2_Dropdown.value.endswith( ".nc" ):
        LoadResults_CDF( SavedFilenames2_Dropdown.value )
    else:
        LoadResults( SavedFilenames2_Dropdown.value )

    for B in Bins:
        if B.JH_mean != 0:
            Means2.append( B.JH_mean )
            StDev2.append( math.sqrt(B.JH_variance) )
    # plot bars chart
    Bars = list()
    fig = go.Figure(data=[
        go.Bar(name='TIEGCM - JH Mean', x=BinIDs, y=Means1, marker_color=ColorTriplet[1], offsetgroup=0),
        go.Bar(name='Orbit  - JH Mean', x=BinIDs, y=Means2, marker_color=ColorTriplet[2], offsetgroup=1),
        #go.Bar(name='JH StDv 1', x=BinIDs, y=StDev1, marker_color="red",  offsetgroup=0, base=Means1),
        #go.Bar(name='JH StDv 2', x=BinIDs, y=StDev2, marker_color="cyan", offsetgroup=1, base=Means2)
    ])
    fig.update_layout(barmode='group', title='Statistics Comparison - '+RegionDescription , plot_bgcolor=ColorTriplet[0], yaxis = dict(showexponent = 'all',exponentformat = 'e'))
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig) 
    print("Comparison plots finished.")

    

# Returns "Winter", "Spring", "Summer" or "Autumn", depending on a UTC timestamp. Works for years 2015-2017
#                                                                                                          1417392000=12/01/2014@12:00am(UTC)  
# 1425168000=03/01/2015@12:00am(UTC) 1433116800=06/01/2015@12:00am(UTC) 1441065600=09/01/2015@12:00am(UTC) 1448928000=12/01/2015@12:00am(UTC)
# 1456790400=03/01/2016@12:00am(UTC) 1464739200=06/01/2016@12:00am(UTC) 1472688000=09/01/2016@12:00am(UTC) 1480550400=12/01/2016@12:00am(UTC)
# 1488326400=03/01/2017@12:00am(UTC) 1496275200=06/01/2017@12:00am(UTC) 1504224000=09/01/2017@12:00am(UTC) 1512086400=12/01/2017@12:00am(UTC)
# 1519862400=03/01/2018@12:00am(UTC)
def getSeason( UTCtimestamp ):
    result = "UTC timestamp exceed range of 20015-2017"
    if   UTCtimestamp>=1417392000 and UTCtimestamp<1425168000: 
        result = "Winter"
    elif UTCtimestamp>=1425168000 and UTCtimestamp<1433116800: 
        result = "Spring"
    elif UTCtimestamp>=1433116800 and UTCtimestamp<1441065600: 
        result = "Summer"        
    elif UTCtimestamp>=1441065600 and UTCtimestamp<1448928000: 
        result = "Autumn"
    elif UTCtimestamp>=1448928000 and UTCtimestamp<1456790400: 
        result = "Winter"
    elif UTCtimestamp>=1456790400 and UTCtimestamp<1464739200: 
        result = "Spring"
    elif UTCtimestamp>=1464739200 and UTCtimestamp<1472688000: 
        result = "Summer"
    elif UTCtimestamp>=1472688000 and UTCtimestamp<1480550400: 
        result = "Autumn"        
    elif UTCtimestamp>=1480550400 and UTCtimestamp<1488326400: 
        result = "Winter"                
    elif UTCtimestamp>=1488326400 and UTCtimestamp<1496275200: 
        result = "Spring"        
    elif UTCtimestamp>=1496275200 and UTCtimestamp<1504224000: 
        result = "Summer"                
    elif UTCtimestamp>=1504224000 and UTCtimestamp<1512086400: 
        result = "Autumn"                
    elif UTCtimestamp>=1512086400 and UTCtimestamp<1519862400: 
        result = "Winter"
    #    
    return result
    
    
'''    
def plotAltitudeProfiles_perSeason( plot_all_seasons_together ):
    if Plot_AltitudeProfiles_Checkbox.value == False: return # <<<
    # init parameters
    if SELECTED_VARIABLE == "Ohmic":
        x_axes_range=[0, 6] # JH
        MultiplicationFactor = 10**8 
        new_units = "10^-8 W/m3"
    elif SELECTED_VARIABLE == "SIGMA_PED":
        x_axes_range=[0, 0.3]
        MultiplicationFactor = 10**3 
        new_units = "mS/m"
    elif SELECTED_VARIABLE == "SIGMA_HAL":
        x_axes_range=[0, 1.5]
        MultiplicationFactor = 10**3 
        new_units = "mS/m"        
    else:
        x_axes_range=[0, 100] 
        MultiplicationFactor = 1
        new_units = "?"
        
    # init data structures
    Profiles = dict()
    MLT_duration_of_a_profile = 3
    ALT_distance_of_a_bucket  = 4
    MLTsequence     = list( range( 0,  24, MLT_duration_of_a_profile) )
    ALTsequence     = list( range(80, 150, ALT_distance_of_a_bucket ) )
    if plot_all_seasons_together:
        SEASONSsequence = [ "All Seasons" ]
    else:
        SEASONSsequence = [ "Spring", "Summer", "Autumn", "Winter" ] 
    if len(SEASONSsequence) > 1: x_axes_range[1] *= 1.4
    for aMLT in MLTsequence:
        for anALT in ALTsequence:
            for aSEASON in SEASONSsequence:
                Profiles[(aSEASON, aMLT, anALT)] = list()
        
    # parse all values and decide into which sum they must fall
    for i in range( 0, len(all_Time_values) ):
        mlt_to_fall = alt_to_fall = -1        
        # find correct season
        if SEASONSsequence[0] == "All Seasons":
            season_to_fall = "All Seasons"
        else:
            season_to_fall = getSeason( all_Time_values[i] )     
        if len(season_to_fall) > 15: print("Error: wrong season for idx =", i, "  UTC =",  all_Time_values[i], season_to_fall )        
        # find correct MLT
        for seq_idx in range(1, len(MLTsequence)):
            if all_MLT_values[i] < MLTsequence[seq_idx]: 
                mlt_to_fall=MLTsequence[seq_idx-1]
                break
        if mlt_to_fall == -1: mlt_to_fall = MLTsequence[len(MLTsequence)-1] # for hour=24                
        # find correct Alt
        for seq_idx in range(1, len(ALTsequence)):
            if all_Altitude_values[i] < ALTsequence[seq_idx]: 
                alt_to_fall=ALTsequence[seq_idx-1]
                break
        if alt_to_fall == -1: continue # ignore highest altitudes
        # store the value at the right place
        Profiles[ (season_to_fall, mlt_to_fall, alt_to_fall) ].append( all_JH_values[ i ] )
    
    # plot
    fig = make_subplots(rows=len(SEASONSsequence), cols=8, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.02, subplot_titles=("0-3", "3-6", "6-9", "9-12", "12-15", "15-18", "18-21", "21-24"))
    for aSEASON in SEASONSsequence:
        for aMLT in MLTsequence:
            Means = list()
            Percentiles10 = list()
            Percentiles25 = list()
            Percentiles50 = list()
            Percentiles75 = list()
            Percentiles90 = list()
            hits  = 0
            for anALT in ALTsequence:
                hits += len(Profiles[(aSEASON, aMLT, anALT)])
                if len(Profiles[(aSEASON, aMLT, anALT)]) > 0:
                    Means.append(  sum(Profiles[(aSEASON, aMLT, anALT)]) / len(Profiles[(aSEASON, aMLT, anALT)]) )
                    Percentiles10.append( np.percentile(Profiles[(aSEASON, aMLT, anALT)], 10) )
                    Percentiles25.append( np.percentile(Profiles[(aSEASON, aMLT, anALT)], 25) )
                    Percentiles50.append( np.percentile(Profiles[(aSEASON, aMLT, anALT)], 50) )
                    Percentiles75.append( np.percentile(Profiles[(aSEASON, aMLT, anALT)], 75) )
                    Percentiles90.append( np.percentile(Profiles[(aSEASON, aMLT, anALT)], 90) )
                else:
                    Means.append( 0 )
                    Percentiles10.append( 0 )
                    Percentiles25.append( 0 )
                    Percentiles50.append( 0 )
                    Percentiles75.append( 0 )
                    Percentiles90.append( 0 )
            print( aSEASON, "MLT =", aMLT, "  Hits =", hits )
            
            # change units
            for i in range(0,len(Means)): 
                Means[i] *= MultiplicationFactor
                Percentiles75[i] *= MultiplicationFactor
                Percentiles90[i] *= MultiplicationFactor
            
            # plot percentiles 10th, 25th, 50th
            fig.add_trace( go.Scatter(x=Percentiles10, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='red', line=dict(color='gray',width=1,), showlegend=False), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles25, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='yellow', line=dict(color='gray',width=1,), showlegend=False), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles50, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='green', line=dict(color='gray',width=1,), showlegend=False), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
            # plot mean
            fig.add_trace( go.Scatter(x=Means, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='#1995ad', line=dict(color='black',width=1,), showlegend=False), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
            # plot percentiles 75th, 90th
            fig.add_trace( go.Scatter(x=Percentiles75, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='#a1d6e2', line=dict(color='gray',width=1,), showlegend=False), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles90, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='#c4dfe6', line=dict(color='gray',width=1,), showlegend=False), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )

    fig.add_trace( go.Scatter(name='Mean value', x=Means, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='#5cc5ef', line=dict(color='black',width=1,), showlegend=True), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )            
    fig.add_trace( go.Scatter(name='75th Percentile', x=Percentiles75, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='#a1d6e2', line=dict(color='gray',width=1,), showlegend=True), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='90th Percentile', x=Percentiles90, y=ALTsequence, mode='lines', fill='tonexty', fillcolor='#c4dfe6', line=dict(color='gray',width=1,), showlegend=True), row=SEASONSsequence.index(aSEASON)+1, col=MLTsequence.index(aMLT)+1 )
            
    #fig.update_yaxes( title="Altitude(km)" )
    for aSEASON in SEASONSsequence:
        fig.update_yaxes( title_text="Altitude (km)", row=SEASONSsequence.index(aSEASON)+1, col=1)
        fig.update_yaxes( title_text=aSEASON, row=SEASONSsequence.index(aSEASON)+1, col=8, side='right' )
    fig.update_xaxes( range=x_axes_range )        
    fig.update_yaxes( range=[80, 150] )  
    fig.update_layout( title = getBinDescription(CALCULATIONS_RegionName) + " - " + "Altitude Profile of " + SELECTED_VARIABLE_longname + " (" + new_units + ")",
                       width=1000, height=200+200*len(SEASONSsequence), showlegend=True) 
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig) 
'''

    
    
    
    
    
    
    
    
    
Profiles = dict()    
MLTsequence = list()
ALTsequence = list()
MLT_duration_of_a_profile = 0
ALT_distance_of_a_bucket = 0
regionMLTmax = 0
regionMLTmin = 0
ProfilesUpdateLock = threading.Lock()   
class Thread_AltProfBinner (threading.Thread):
    def __init__(self, from_idx, to_idx):
        threading.Thread.__init__(self)
        self.from_idx = from_idx
        self.to_idx = to_idx
    def run(self):
        global Profiles
        for i in range( self.from_idx, self.to_idx ):
            mlt_to_fall = alt_to_fall = -1  
            # find correct Alt
            for seq_idx in range(0, len(ALTsequence)):
                if all_Altitude_values[i]>=ALTsequence[seq_idx] and all_Altitude_values[i]<ALTsequence[seq_idx]+ALT_distance_of_a_bucket:
                    alt_to_fall=ALTsequence[seq_idx]
                    break
            if alt_to_fall == -1: continue # ignore highest altitudes        
            # find correct kp
            if all_Kp_values[i] < 2: 
                kp_to_fall = 0
            elif all_Kp_values[i] < 4:  
                kp_to_fall = 2
            else:
                kp_to_fall = 4
            # find correct MLT
            MLT_tocheck = all_MLT_values[i]
            if regionMLTmax>24  and  MLT_tocheck<=regionMLTmax-regionMLTmin:
                MLT_tocheck += 24
            for seq_idx in range(0, len(MLTsequence)):
                if MLT_tocheck>=MLTsequence[seq_idx] and MLT_tocheck<MLTsequence[seq_idx]+MLT_duration_of_a_profile: 
                    mlt_to_fall=MLTsequence[seq_idx]
                    break
            if MLT_tocheck == MLTsequence[len(MLTsequence)-1]+MLT_duration_of_a_profile: mlt_to_fall = MLTsequence[len(MLTsequence)-1] # for last MLT position
            # store the value at the right place
            #ProfilesUpdateLock.acquire()
            Profiles[ (kp_to_fall, mlt_to_fall, alt_to_fall) ].append( all_JH_values[ i ] )
            #ProfilesUpdateLock.release()   

    
def plotAltitudeProfiles_perKpRange( ):
    global Profiles, MLTsequence, ALTsequence, MLT_duration_of_a_profile, ALT_distance_of_a_bucket, regionMLTmax, regionMLTmin
    if Plot_AltitudeProfiles_Checkbox.value == False: return # <<<
    # init parameters
    if SELECTED_VARIABLE == "Ohmic":
        x_axes_range=[0, 6]
        MultiplicationFactor = 10**8 
        new_units = "10^-8 W/m3"
    elif SELECTED_VARIABLE == "SIGMA_PED":
        x_axes_range=[0, 0.15]
        MultiplicationFactor = 10**3 
        new_units = "mS/m"
    elif SELECTED_VARIABLE == "SIGMA_HAL":
        x_axes_range=[0, 0.4] 
        MultiplicationFactor = 10**3 
        new_units = "mS/m"        
    elif SELECTED_VARIABLE == "Convection_heating":
        x_axes_range=[0, 6] 
        MultiplicationFactor = 10**8 
        new_units = "10^-8 W/m3"           
    elif SELECTED_VARIABLE == "Wind_heating":
        x_axes_range=[0, 6] 
        MultiplicationFactor = 10**8 
        new_units = "10^-8 W/m3"                   
    elif SELECTED_VARIABLE == "EEX_si" or SELECTED_VARIABLE == "EEY_si":
        x_axes_range=[-24, 0] 
        MultiplicationFactor = 1
        new_units = "mV/m"        
    else:
        x_axes_range=[0, 100] 
        MultiplicationFactor = 1
        new_units = "?" 
        
    print(SELECTED_VARIABLE)

    # Region specific binning:
    if "(" in CALCULATIONS_RegionName:
        RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    else:
        RegionID = CALCULATIONS_RegionName
    regionMLTmin = 999
    regionMLTmax = -999
    regionALTmin = 999
    regionALTmax = -999
    for B in Bins:
        if B.ID.startswith( RegionID ):
            if regionMLTmin>B.MLT_min: regionMLTmin = B.MLT_min
            if regionMLTmax<B.MLT_max: regionMLTmax = B.MLT_max
            if regionALTmin>B.Altitude_min: regionALTmin = B.Altitude_min
            if regionALTmax<B.Altitude_max: regionALTmax = B.Altitude_max
    if regionMLTmax < regionMLTmin: regionMLTmax += 24
    # init data structures
    Profiles = dict()
    if "TRO" in RegionID:
        MLT_duration_of_a_profile = 3        
    else:
        MLT_duration_of_a_profile = 6
    ALT_distance_of_a_bucket  = 5
    MLTsequence     = list( range( regionMLTmin,  regionMLTmax, MLT_duration_of_a_profile) )
    ALTsequence     = list( range( regionALTmin, regionALTmax, ALT_distance_of_a_bucket ) )
    KPsequence      = [ 0, 2, 4 ] 
    for aMLT in MLTsequence:
        for anALT in ALTsequence:
            for aKP in KPsequence:
                Profiles[(aKP, aMLT, anALT)] = list()
    
    print( "Parsing", len(all_JH_values), "values.", datetime.now().strftime("%d-%m-%Y %H:%M:%S") )
    # parse all values and decide into which sum they must fall
    AllThreads = list()
    positions_per_thread = int ( len(all_JH_values) / 10 )
    from_pos = 0
    while from_pos < len(all_JH_values):
        # calculate boundaries for thread
        to_pos = from_pos + positions_per_thread
        if to_pos >= len(all_JH_values): to_pos = len(all_JH_values)-1
        if len(all_JH_values)-to_pos<positions_per_thread : to_pos = len(all_JH_values)-1
        # spawn new thread
        print("Thread:", from_pos, "-", to_pos, " of " ,len(all_JH_values), "positions")
        T = Thread_AltProfBinner(from_pos, to_pos)
        AllThreads.append(T)
        T.start()
        # go on
        from_pos += positions_per_thread
        
    # wait for all threads to terminate
    for T in AllThreads: T.join()

    # plot
    Color10 = '#c4dfe6'
    Color25 = '#a1d6e2'
    Color50 = '#1995ad'
    Color75 = '#a1d6e2'
    Color90 = '#c4dfe6'
    
    # construct the column MLT titles #("0-3", "3-6", "6-9", "9-12", "12-15", "15-18", "18-21", "21-24")
    ColumnTitles = list()
    
    for i in range(0, len(MLTsequence)):
        ColumnTitles.append( "MLT " + str(MLTsequence[i]) + "-"  + str(MLTsequence[i]+MLT_duration_of_a_profile) )
    # define secondary y-axis at the right of the plot
    mySpecs = list()
    for row in range(0, len(KPsequence)):
        mySpecs.append( list() )
        for col in range(0, len(MLTsequence)):
            mySpecs[row].append( {"secondary_y": True} )
        
    #make plot
    fig = make_subplots(rows=len(KPsequence), cols=len(MLTsequence), shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.035, horizontal_spacing=0.01, subplot_titles=ColumnTitles, specs=mySpecs)
    for aKP in KPsequence:
        for aMLT in MLTsequence:
            #Means = list()
            Percentiles10 = list()
            Percentiles25 = list()
            Percentiles50 = list()            
            Percentiles75 = list()
            Percentiles90 = list()
            visibleALTsequence = list()
            hits  = 0
            for anALT in ALTsequence:
                print("  ", anALT, "km     hits =",  len(Profiles[(aKP, aMLT, anALT)]))
                hits += len(Profiles[(aKP, aMLT, anALT)])
                if len(Profiles[(aKP, aMLT, anALT)]) > 0:
                    #Means.append(  sum(Profiles[(aKP, aMLT, anALT)]) / len(Profiles[(aKP, aMLT, anALT)]) )
                    Percentiles10.append( np.percentile(Profiles[(aKP, aMLT, anALT)], 10) )
                    Percentiles25.append( np.percentile(Profiles[(aKP, aMLT, anALT)], 25) )
                    Percentiles50.append( np.percentile(Profiles[(aKP, aMLT, anALT)], 50) )                    
                    Percentiles75.append( np.percentile(Profiles[(aKP, aMLT, anALT)], 75) )
                    Percentiles90.append( np.percentile(Profiles[(aKP, aMLT, anALT)], 90) )
                    visibleALTsequence.append( anALT )
                #else:
                    #Means.append( 0 )
                    #Percentiles10.append( 0 )
                    #Percentiles25.append( 0 )
                    #Percentiles50.append( 0 )                    
                    #Percentiles75.append( 0 )
                    #Percentiles90.append( 0 )
            print( "Kp = ", aKP, "MLT =", aMLT, "   Hits =", hits, "  ", datetime.now().strftime("%d-%m-%Y %H:%M:%S") )
            
            # change units
            for i in range(0,len(Percentiles50)): 
                #Means[i] *= MultiplicationFactor
                Percentiles10[i] *= MultiplicationFactor
                Percentiles25[i] *= MultiplicationFactor
                Percentiles50[i] *= MultiplicationFactor
                Percentiles75[i] *= MultiplicationFactor
                Percentiles90[i] *= MultiplicationFactor
            
            # alter visibleALTsequence so that data are displayed correctly
            for i in range(0, len(visibleALTsequence)):
                visibleALTsequence[i] += ALT_distance_of_a_bucket/2
            visibleALTsequence[0]  = regionALTmin
            visibleALTsequence[-1] = regionALTmax
            
            fig.add_trace( go.Scatter(x=[0]*len(visibleALTsequence), y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles10, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles25, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color25, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles50, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color50, line=dict(color='black',width=2,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            # plot mean
            #fig.add_trace( go.Scatter(x=Means, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor='black', line=dict(color='black',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            # plot percentiles
            fig.add_trace( go.Scatter(x=Percentiles75, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color75, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles90, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color90, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1,  )
            # add a trace in order to display secondary y-axis at the right
            fig.add_trace( go.Scatter(x=[-1000], y=[-1000], showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, secondary_y=True )
            
    # display legends
    fig.add_trace( go.Scatter(name='10th Perc.', x=Percentiles10, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='25th Perc.', x=Percentiles25, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color25, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='50th Perc.', x=Percentiles50, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color50, line=dict(color='black',width=2,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    #fig.add_trace( go.Scatter(name='Mean value', x=Means, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor='#5cc5ef', line=dict(color='black',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )            
    fig.add_trace( go.Scatter(name='75th Perc.', x=Percentiles75, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color75, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='90th Perc.', x=Percentiles90, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color90, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    
    
    #fig.update_yaxes( title="Altitude(km)" )
    for aKP in KPsequence:
        fig.update_yaxes( title_text="Altitude (km)", row=KPsequence.index(aKP)+1, col=1, side='left', secondary_y=False)
        row_title = "Kp " + str(aKP) + " - "
        if aKP == 0:
            row_title +=  "2"
        elif aKP == 2:
            row_title +=  "4"
        else:
            row_title +=  "9"
        fig.update_yaxes( title_text=row_title, row=KPsequence.index(aKP)+1, col=len(MLTsequence),  side='right', secondary_y=True, showticklabels=False )
        for aMLT in MLTsequence:
            fig.update_yaxes( row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, secondary_y=True, showticklabels=False )
    fig.update_xaxes( range=x_axes_range )
    fig.update_yaxes( range=[80, 150], dtick=10 )  
    fig.update_layout( title = getBinDescription(CALCULATIONS_RegionName) + "<br>" + "Alt.Prof. of " + SELECTED_VARIABLE_longname + " (" + new_units + ")",
                       width=280+len(MLTsequence)*100, height=200+200*len(KPsequence), showlegend=True, legend_orientation="h", legend_y=-0.04) 
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig) 
    
    # plot more zoom versions
    new_x_axes_range = [x * (2/3) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (1/2) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (3/2) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (2.5) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    
    
    
DiskAccessLock = threading.Lock()   
class Thread_ValueAssigner (threading.Thread):
    def __init__(self, DataFilename, ResultsFilename):
        threading.Thread.__init__(self)
        self.DataFilename = DataFilename
        self.ResultsFilename = ResultsFilename
    def run(self):
        DataFilename = self.DataFilename
        ResultsFilename = self.ResultsFilename
        print( "Thread start",  datetime.now().strftime("%d-%m-%Y %H:%M:%S"), ResultsFilename, "\n" )
        MagLat_min =  1000
        MagLat_max = -1000
        MLT_min    =  1000
        MLT_max    = -1000
        Altitude_min    =  1000
        Altitude_max    = -1000
        Lat_min     =  1000
        Lat_max     = -1000    
        Kp_min     =  1000
        Kp_max     = -1000
        localBins = copy.deepcopy(Bins)
        for B in localBins:
            B.reset()
            if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
            if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
            if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
            if B.MLT_max > MLT_max: MLT_max = B.MLT_max
            if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
            if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
            if B.Lat_min < Lat_min: Lat_min = B.Lat_min 
            if B.Lat_max > Lat_max: Lat_max = B.Lat_max                        
            if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
            if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        all_JH_values       = list()
        all_MagLat_values   = list() 
        all_MLT_values      = list() 
        all_Altitude_values = list() 
        all_Lat_values      = list()
        all_Kp_values       = list() 
        all_Time_values     = list()
        all_HittedBin_IDs   = list()
        all_EEX_values      = list()
        all_EEY_values      = list()
        all_Pedersen_values = list()
        all_Hall_values     = list()
        all_ConvectionHeating_values = list()
        all_WindHeating_values = list()
        Matches = 0
        
        # parse TIEGCM file
        try:
            CDFroot = Dataset( DataFilename, 'r' )
        except:
            print ( "WRONG FORMAT:", DataFilename )
            return
        try:
            FileStartTimeStamp = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"
        except:
            print ( "WRONG CONTENTS:", DataFilename )
            return
        length_time = CDFroot.variables['Ohmic'].shape[0]
        length_lev  = CDFroot.variables['Ohmic'].shape[1]
        length_lat  = CDFroot.variables['Ohmic'].shape[2]
        length_lon  = CDFroot.variables['Ohmic'].shape[3]
        # wait until disk is released
        DiskAccessLock.acquire()
        # Load or calculate all basic values from the netcdf file
        try:
            TIMEs   = CDFroot.variables['time'][:] # minutes since the start time
            LATs    = CDFroot.variables['lat'][:] 
            ALTs    = CDFroot.variables['ZGMID'][:, :, :, :] / 100000 # it is stored in cm inside the file
            JHs     = CDFroot.variables['Ohmic'][:, :, :, :]
            KPs     = CDFroot.variables['Kp'][:]
            MAGLATs = CDFroot.variables['mlat_qdf'][:, :, :, :] 
            MLTs    = CDFroot.variables['mlt_qdf'][:, :, :, :] 
            EEXs    = CDFroot.variables['EEX_si'][:, :, :, :] 
            EEYs    = CDFroot.variables['EEY_si'][:, :, :, :] 
            PEDs    = CDFroot.variables['SIGMA_PED'][:, :, :, :] 
            HALs    = CDFroot.variables['SIGMA_HAL'][:, :, :, :]
            try:
                CONV_H  = CDFroot.variables['Convection_heating'][:, :, :, :]
            except:
                CONV_H  = CDFroot.variables['Convenction_heating'][:, :, :, :]
            WIND_H  = CDFroot.variables['Wind_heating'][:, :, :, :]
        except:
            print( "Thread aborted while reading",  datetime.now().strftime("%d-%m-%Y %H:%M:%S"), ResultsFilename[-22:], "\n" )
            DiskAccessLock.release()
            return 
        DiskAccessLock.release()
        print( "Thread file read done",  datetime.now().strftime("%d-%m-%Y %H:%M:%S"), ResultsFilename[-22:], "\n" )
    
        step = 1
        for idx_lat in range(0, length_lat, step):
            if idx_lat%30==0: print("Thread Calculating Lat",  idx_lat, ResultsFilename[-22:])
            current_Lat = LATs[idx_lat] 
            if current_Lat < Lat_min  or  current_Lat > Lat_max: continue
            for idx_lon in range(0, length_lon, step):
                for idx_lev in range(0, length_lev, step):
                    for idx_time in range(0, length_time, step):                    
                        in_Altitude_range = in_MagLat_range = in_MLT_range = in_Kp_range = False
                            
                        current_Altitude = ALTs[idx_time, idx_lev, idx_lat, idx_lon]
                        if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max:
                            in_Altitude_range = True
                        
                        if in_Altitude_range:
                            current_MagLat = MAGLATs[ idx_time, idx_lev, idx_lat, idx_lon ]
                            if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max:
                                in_MagLat_range = True
                                
                        if in_MagLat_range:
                            current_MLT = MLTs[ idx_time, idx_lev, idx_lat, idx_lon ]
                            if in_MagLat_range:
                                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )
                        
                        if in_MLT_range:
                            current_Kp = KPs[idx_time]
                            if current_Kp >= Kp_min and current_Kp <= Kp_max:
                                in_Kp_range = True   
                                
                        if in_Kp_range:                    
                            matchedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp, current_Lat )
                            if matchedBin is not None:
                                for B in localBins:
                                    if B.ID == matchedBin.ID:
                                        matchedBin = B
                                current_time = int( FileStartTimeStamp + TIMEs[idx_time]*120*60 )
                                current_JH = JHs[idx_time, idx_lev, idx_lat ,idx_lon] #CDFroot.variables['Joule Heating'][idx_time, idx_lev, idx_lat, idx_lon]
                                matchedBin.JH_values.append( current_JH )
                                matchedBin.MagLat_values.append( current_MagLat )
                                matchedBin.MLT_values.append( current_MLT )
                                matchedBin.Altitude_values.append( current_Altitude )
                                matchedBin.Kp_values.append( current_Kp )
                                matchedBin.Time_values.append( current_time )
                                matchedBin.EEX_values.append( EEXs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.EEY_values.append( EEYs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.Pedersen_values.append( PEDs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.Hall_values.append( HALs[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.ConvectionHeating_values.append( CONV_H[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                matchedBin.WindHeating_values.append( WIND_H[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                all_JH_values.append( current_JH )
                                all_MagLat_values.append( current_MagLat )
                                all_MLT_values.append( current_MLT )
                                all_Altitude_values.append( current_Altitude )
                                all_Kp_values.append( current_Kp )
                                all_Time_values.append( current_time )
                                all_HittedBin_IDs.append( matchedBin.ID )
                                all_EEX_values.append( EEXs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_EEY_values.append( EEYs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_Pedersen_values.append( PEDs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_Hall_values.append( HALs[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                all_ConvectionHeating_values.append( CONV_H[ idx_time, idx_lev, idx_lat, idx_lon ] ) 
                                all_WindHeating_values.append( WIND_H[ idx_time, idx_lev, idx_lat, idx_lon ] )
                                Matches += 1
                    #break
                #break
        CDFroot.close()
        # wait until disk is released
        #DiskAccessLock.acquire()
        #### SAVE Results ####
        try:
            # save general info
            resultsCDF = Dataset( ResultsFilename, 'a' )
            resultsCDF.DateOfUpdate = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            resultsCDF.Region = CALCULATIONS_RegionName
            resultsCDF.DataPath = CALCULATIONS_TIEGCMfolder
            # save data for each bin seperately 
            for B in localBins:
                # save data about the hits inside the bin
                if len(B.Time_values) > 0:
                    resultsCDF.variables[B.ID+"_TimeValues"][:]      = B.Time_values
                    resultsCDF.variables[B.ID+"_JHValues"][:]        = B.JH_values        
                    resultsCDF.variables[B.ID+"_MagLatValues"][:]    = B.MagLat_values
                    resultsCDF.variables[B.ID+"_MLTValues"][:]       = B.MLT_values
                    resultsCDF.variables[B.ID+"_AltitudeValues"][:]  = B.Altitude_values
                    resultsCDF.variables[B.ID+"_LatValues"][:]       = B.Lat_values
                    resultsCDF.variables[B.ID+"_KpValues"][:]        = B.Kp_values
                    resultsCDF.variables[B.ID+"_EEXValues"][:]       = B.EEX_values        
                    resultsCDF.variables[B.ID+"_EEYValues"][:]       = B.EEY_values
                    resultsCDF.variables[B.ID+"_PedersenValues"][:]  = B.Pedersen_values
                    resultsCDF.variables[B.ID+"_HallValues"][:]      = B.Hall_values
                    resultsCDF.variables[B.ID+"_ConvectionHeatingValues"][:] = B.ConvectionHeating_values
                    resultsCDF.variables[B.ID+"_WindHeatingValues"][:] = B.WindHeating_values
            ## save data for all hits
            resultsCDF.variables["allTimeValues"][:]     = all_Time_values
            resultsCDF.variables["allJHValues"][:]       = all_JH_values    
            resultsCDF.variables["allMagLatValues"][:]   = all_MagLat_values
            resultsCDF.variables["allMLTValues"][:]      = all_MLT_values
            resultsCDF.variables["allAltitudeValues"][:] = all_Altitude_values
            resultsCDF.variables["allLatValues"][:]      = all_Lat_values
            resultsCDF.variables["allKpValues"][:]       = all_Kp_values
            #resultsCDF.variables["allHittedBinIDs"][:]   = netCDF4.stringtochar(np.array(all_HittedBin_IDs[:], 'S8'))
            resultsCDF.variables["allEEXValues"][:]      = all_EEX_values
            resultsCDF.variables["allEEYValues"][:]      = all_EEY_values
            resultsCDF.variables["allPedersenValues"][:] = all_Pedersen_values
            resultsCDF.variables["allHallValues"][:]     = all_Hall_values
            resultsCDF.variables["allConvectionHeatingValues"][:] = all_ConvectionHeating_values
            resultsCDF.variables["allWindHeatingValues"][:] = all_WindHeating_values
            #
            resultsCDF.close()    
        except Exception as e:
            print( "!!!! Thread error while writing",  datetime.now().strftime("%d-%m-%Y %H:%M:%S"), ResultsFilename[-22:], "\n" )
            print( e )
            #DiskAccessLock.release()
        #DiskAccessLock.release()
    
        print( "Thread finish",  datetime.now().strftime("%d-%m-%Y %H:%M:%S"), ResultsFilename[-22:], "\n", Matches, "matches", len(localBins[0].JH_values), len(Bins[0].JH_values) )
        print( "" )
    
    
def AssignValuesPerBin_MultipleResultFiles( DataFilesPath ):
    startSecs = time.time()

    ResultsFolder = DaedalusGlobals.CoverageResults_Files_Path + BinGroups_Dropdown.value + "." + CALCULATIONS_TIEGCMfolder[CALCULATIONS_TIEGCMfolder[:-1].rfind('/')+1:-1] + ".MultiFileResults/"
    CALCULATIONS_ResultsFolder = ResultsFolder
    if path.exists( ResultsFolder ) == False:
        os.mkdir( ResultsFolder )
    
    AllThreads = list()
    AllDataFiles = sorted( glob.glob( DataFilesPath + "TIEGCM*/*.nc", recursive=True ) )
    for currentDataFile in AllDataFiles:
        ResultsFilename = ResultsFolder + currentDataFile[ currentDataFile.rfind('/')+1 : -3 ] + ".nc"
        if path.exists( ResultsFilename ): 
            print("Skipping because exists:", ResultsFilename)
            continue
        else:
            # wait if there are plenty alive threads
            alive_counter = 0
            for aThread in AllThreads:
                if aThread.is_alive():
                    alive_counter += 1
            while alive_counter >= 36:
                time.sleep(random.randint(10, 15))
                alive_counter = 0
                for aThread in AllThreads:
                    if aThread.is_alive():
                        alive_counter += 1
            # spawn new thread
            CreateResults_CDF( ResultsFilename )
            T = Thread_ValueAssigner(currentDataFile, ResultsFilename)
            AllThreads.append(T)
            T.start()
            time.sleep(2)

    # wait for all threads to terminate
    for T in AllThreads: T.join()
    # finish it
    finishSecs = time.time()
    print( finishSecs-startSecs, " sec")    
    return ResultsFolder
    
    
    
    
    
    
    
def plotHeightIntegrated_perKpRange():
    print( "Height-integration plot started", datetime.now().strftime("%d-%m-%Y %H:%M:%S") )
    # init parameters
    if SELECTED_VARIABLE == "Ohmic":
        SELECTED_VARIABLE_longname = "Joule Heating"
        MultiplicationFactor = 1000
        new_units = "mW/m3"
    elif SELECTED_VARIABLE == "SIGMA_PED":
        SELECTED_VARIABLE_longname = "Pedersen Conductivity"
        MultiplicationFactor = 1
        new_units = "S/m"
    elif SELECTED_VARIABLE == "SIGMA_HAL":
        SELECTED_VARIABLE_longname = "Hall Conductivity"
        MultiplicationFactor = 1
        new_units = "S/m"        
    elif SELECTED_VARIABLE == "Convection_heating":
        SELECTED_VARIABLE_longname = "Convection Heating"
        MultiplicationFactor = 1000
        new_units = "mW/m3"           
    elif SELECTED_VARIABLE == "Wind_heating":
        SELECTED_VARIABLE_longname = "Wind Heating"
        MultiplicationFactor = 1000
        new_units = "mW/m3"                   
    elif SELECTED_VARIABLE == "EEX_si" or SELECTED_VARIABLE == "EEY_si":
        MultiplicationFactor = 1
        new_units = "mV/m"        
    else:
        MultiplicationFactor = 1
        new_units = "?" 
        
    print( "Variable" , SELECTED_VARIABLE)

    # Region specific binning:
    s = SavedFilenames_Dropdown.value[:-1]
    s = s[ s.rfind('/')+1 : ]
    RegionID = s[ 0 : s.find('.')]
    print( "Region", RegionID )
    regionMagLatmin = 999
    regionMagLatmax = -999
    regionMLTmin = 999
    regionMLTmax = -999
    regionLATmin = 999
    regionLATmax = -999
    for B in Bins:
        if B.ID.startswith( RegionID ):
            if regionMagLatmin>B.MagLat_min: regionMagLatmin = B.MagLat_min
            if regionMagLatmax<B.MagLat_max: regionMagLatmax = B.MagLat_max            
            if regionMLTmin>B.MLT_min: regionMLTmin = B.MLT_min
            if regionMLTmax<B.MLT_max: regionMLTmax = B.MLT_max
            if regionLATmin>B.Lat_min: regionLATmin = B.Lat_min
            if regionLATmax<B.Lat_max: regionLATmax = B.Lat_max
    if regionMLTmax < regionMLTmin: regionMLTmax += 24
        
    # init data structures
    MLTsequence     = list( range( regionMLTmin,  regionMLTmax, 1) )
    KPsequence      = [ 0, 2, 4 ] 
    Distribution    = dict()
    for aKP in KPsequence:
        for aMLT in MLTsequence:
            Distribution[ (aKP, aMLT) ] = list()
        
    # read data and put them into the data structures
    AllDataFilenames = sorted( glob.glob("/home/NAS/TIEGCM_DATA_2/Height_Integrated_Products/*.nc") )
    for file_idx in range(0, len(AllDataFilenames)):
        if file_idx%10==0: print( "Reading", AllDataFilenames[file_idx] )
        CDFroot = Dataset( AllDataFilenames[file_idx], 'r' )
        length_time = CDFroot.variables['mlt_qdf'].shape[0]
        length_lat  = CDFroot.variables['mlt_qdf'].shape[1]
        length_lon  = CDFroot.variables['mlt_qdf'].shape[2]
        MAGLAT_values = CDFroot.variables['mlat_qdf'][:, :, :] 
        MLT_values    = CDFroot.variables['mlt_qdf'][:, :, :] 
        LAT_values    = CDFroot.variables['lat'][:] 
        KP_values     = CDFroot.variables['Kp'][:] 
        if SELECTED_VARIABLE == "Ohmic":
            try:
                VAR_values = CDFroot.variables['Convection_heating'][:, :, :] + CDFroot.variables['Wind_heating'][:, :, :]
            except:
                VAR_values = CDFroot.variables['Convenction_heating'][:, :, :] + CDFroot.variables['Wind_heating'][:, :, :]
        elif SELECTED_VARIABLE == "SIGMA_PED":
            VAR_values = CDFroot.variables['SigmaP_HI'][:, :, :] 
        elif SELECTED_VARIABLE == "SIGMA_HAL":
            VAR_values = CDFroot.variables['SigmaH_HI'][:, :, :] 
        elif SELECTED_VARIABLE == "Convection_heating":
            try:
                VAR_values = CDFroot.variables['Convection_heating'][:, :, :] 
            except:
                VAR_values = CDFroot.variables['Convenction_heating'][:, :, :] 
        elif SELECTED_VARIABLE == "Wind_heating":
            VAR_values = CDFroot.variables['Wind_heating'][:, :, :] 
        VAR_values *= MultiplicationFactor
        CDFroot.close()
        # parse data into the structrures
        for idx_time in range(0, length_time, 1):
            for idx_lat in range(0, length_lat, 1):
                for idx_lon in range(0, length_lon, 1):
                    in_MagLat_range = in_MLT_range = in_Lat_range = False
                    
                    currentMagLat = MAGLAT_values[ idx_time, idx_lat, idx_lon ]
                    if currentMagLat>=regionMagLatmin and currentMagLat<=regionMagLatmax: in_MagLat_range = True
                        
                    if in_MagLat_range:
                        currentLAT = LAT_values[ idx_lat ]    
                        if currentLAT>=regionLATmin and currentLAT<=regionLATmax: in_Lat_range = True
                        
                    if in_Lat_range:
                        currentMLT = MLT_values[ idx_time, idx_lat, idx_lon ]
                        MLT_toCheck = currentMLT
                        if regionMLTmax > 24  and  currentMLT<=regionMLTmax-regionMLTmin: MLT_toCheck += 24
                        if MLT_toCheck>=regionMLTmin and MLT_toCheck<=regionMLTmax: in_MLT_range = True
                            
                    if in_MLT_range:
                        currentKP = KP_values[ idx_time ]
                        # find correct kp
                        if currentKP < 2: 
                            kp_to_fall = 0
                        elif currentKP < 4:  
                            kp_to_fall = 2
                        else:
                            kp_to_fall = 4
                        # find correct MLT
                        for seq_idx in range(0, len(MLTsequence)):
                            if MLT_toCheck>=MLTsequence[seq_idx] and MLT_toCheck<MLTsequence[seq_idx]+1: 
                                mlt_to_fall=MLTsequence[seq_idx]
                                break
                        if MLT_toCheck == MLTsequence[len(MLTsequence)-1]+1: mlt_to_fall = MLTsequence[len(MLTsequence)-1] # for last MLT position
                        # store the value at the right place
                        Distribution[ (kp_to_fall, mlt_to_fall) ].append( VAR_values[ idx_time, idx_lat, idx_lon ] )
        #break # <<< zoro
        
    Color10 = '#c4dfe6'
    Color25 = '#a1d6e2'
    Color50 = '#1995ad'
    Color75 = '#a1d6e2'
    Color90 = '#c4dfe6'
    # define secondary y-axis at the right of the plot
    mySpecs = list()
    for row in range(0, len(KPsequence)):
        mySpecs.append( {"secondary_y": True} )
    # init figure            
    fig = make_subplots(rows=len(KPsequence), cols=1, shared_xaxes=True, vertical_spacing=0.03, specs=[[{"secondary_y": True}]]*len(KPsequence))

    # calculate Percentiles
    hits = 0
    Ymax = -10000
    for aKP in KPsequence:
        Percentiles10 = list()
        Percentiles25 = list()
        Percentiles50 = list()
        Percentiles75 = list()
        Percentiles90 = list()
        Xaxis_values= list()
        for aMLT in MLTsequence:
            hits += len(Distribution[(aKP, aMLT)])
            print( "Kp = ", aKP, "MLT =", aMLT, "   Hits =", len(Distribution[(aKP, aMLT)]) )
            Xaxis_values.append( aMLT )
            Xaxis_values.append( aMLT + 1 )
            if len(Distribution[(aKP, aMLT)]):
                n = np.percentile(Distribution[(aKP, aMLT)], 10)
                Percentiles10.append( n )
                Percentiles10.append( n )
                n = np.percentile(Distribution[(aKP, aMLT)], 25)
                Percentiles25.append( n )
                Percentiles25.append( n )
                n = np.percentile(Distribution[(aKP, aMLT)], 50)
                Percentiles50.append( n )
                Percentiles50.append( n )
                n = np.percentile(Distribution[(aKP, aMLT)], 75)
                Percentiles75.append( n )
                Percentiles75.append( n )
                n = np.percentile(Distribution[(aKP, aMLT)], 90)
                Percentiles90.append( n )
                Percentiles90.append( n )
        # add traces for percentiles
        DisplayThisLegend = False
        if KPsequence.index(aKP) == 0: DisplayThisLegend = True
        fig.add_trace( go.Scatter(x=Xaxis_values, y=[0]*len(Percentiles10), mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=1 )
        fig.add_trace( go.Scatter(name='10th Perc.', x=Xaxis_values, y=Percentiles10, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray', width=1,), showlegend=DisplayThisLegend), row=KPsequence.index(aKP)+1, col=1 )
        fig.add_trace( go.Scatter(name='25th Perc.', x=Xaxis_values, y=Percentiles25, mode='lines', fill='tonexty', fillcolor=Color25, line=dict(color='gray', width=1,), showlegend=DisplayThisLegend), row=KPsequence.index(aKP)+1, col=1 )
        fig.add_trace( go.Scatter(name='50th Perc.', x=Xaxis_values, y=Percentiles50, mode='lines', fill='tonexty', fillcolor=Color50, line=dict(color='black',width=2,), showlegend=DisplayThisLegend), row=KPsequence.index(aKP)+1, col=1 )
        fig.add_trace( go.Scatter(name='75th Perc.', x=Xaxis_values, y=Percentiles75, mode='lines', fill='tonexty', fillcolor=Color75, line=dict(color='gray', width=1,), showlegend=DisplayThisLegend), row=KPsequence.index(aKP)+1, col=1 )
        fig.add_trace( go.Scatter(name='90th Perc.', x=Xaxis_values, y=Percentiles90, mode='lines', fill='tonexty', fillcolor=Color90, line=dict(color='gray', width=1,), showlegend=DisplayThisLegend), row=KPsequence.index(aKP)+1, col=1 )
        # add a trace in order to display secondary y-axis at the right
        fig.add_trace( go.Scatter(x=[-10], y=[-10], showlegend=False, marker_size=0), row=KPsequence.index(aKP)+1, col=1, secondary_y=True )
        #
        if Ymax < max(Percentiles90): Ymax = max(Percentiles90)
    Ymax = Ymax * 1.05
    print( "Total Hits =", hits, "   Ymax =", Ymax )
        
    # set layout
    for aKP in KPsequence:
        fig.update_yaxes( title_text=new_units, row=KPsequence.index(aKP)+1, col=1, side='left', secondary_y=False)
        row_title = "Kp " + str(aKP) + " - "
        if aKP == 0:
            row_title +=  "2"
        elif aKP == 2:
            row_title +=  "4"
        else:
            row_title +=  "9"
        fig.update_yaxes( title_text=row_title, row=KPsequence.index(aKP)+1, col=1,  side='right', secondary_y=True, showticklabels=False )
        ####
    fig.update_xaxes( range=[MLTsequence[0],MLTsequence[-1]+1], dtick=1 )
    fig.update_xaxes( title_text="Magnetic Local Time (hours)", row=len(KPsequence), col=1 )
    fig.update_yaxes( range=[0, Ymax ] )
    fig.update_layout( title = getBinDescription(RegionID) + "<br>" + "Height-integrated Dsitribution of " + SELECTED_VARIABLE_longname + " (" + new_units + ")",
                       width=800, height=200+200*len(KPsequence), showlegend=True, legend_orientation="h", legend_y=-0.06) 
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)
    print( "Height-integration plot finished", datetime.now().strftime("%d-%m-%Y %H:%M:%S") )
    
    
    
    
    
display( createGUI() )
Plot_JHvsMagLat_Checkbox.value = False
Plot_JHvsMLT_Checkbox.value = False
Plot_JHvsAltitude_Checkbox.value = False
Plot_AltitudeVsMagLat_Checkbox.value = False
Plot_JHdistribution_Checkbox.value = False
#Plot_AltitudeProfiles_Checkbox.value = False
Plot_HeightIntegrated_Checkbox.value = False
#SavedFilenames_Dropdown.value = "/home/NAS/Data_Files/CoverageResults/TRO.TIEGCM_Lifetime_2015_to_2018_JH_QD.TIEGCM_Lifetime_2015_to_2018_CAMP03.ValuesPerBinResults.nc"
SavedFilenames_Dropdown.value = "/home/NAS/Data_Files/CoverageResults/old_noSubBins.TRO.TIEGCM_Lifetime_2015_to_2018_JH_QD.TIEGCM_Lifetime_2015_to_2018_CAMP03.ValuesPerBinResults.nc"
SavedFilenames_Dropdown.value = "/home/balukid/old_onlyOhmic.TRO.TIEGCM_Lifetime_2015_to_2018_JH_QD.MultiFileResults/"
SavedFilenames_Dropdown.value = "/home/NAS/Data_Files/CoverageResults/AEM.TIEGCM_Lifetime_2015_to_2018_JH_QD.MultiFileResults/"
BinGroups_Dropdown.value = "TRO"









